# <font color='#000000'>Master Thesis Hugo Alves | <span style="color:#BFD62F;">__Nova__</span> <span style="color:#5C666C;">__IMS__</span></font>

Welcome to the second notebook developed for this project. We (Professor Roberto Henriques, Professor Ricardo Santos, and Hugo Alves) aim to develop a machine learning (ML) framework to predict student admissions to postgraduate and masters' programs at Nova IMS, as well as the final grade point average (GPA) of those who are accepted.

<br>
<div align="center">
  <img src="https://i.ibb.co/5WGmxWv/Notebooks-Workflow.png" alt="Workflow" width="800" />
</div>
<br>

After our initial exploration of the available data, we can now shift our focus to __data preparation and preprocessing__. Here, we will deal with the transformations that are applicable to both predictive tasks. Enjoy!


# <font color="#5C666C">Contents</font> <a class="anchor" id="toc"></a>
[Initial Setup](#setup)<br>
- [ Library and Functions Import](#library)<br>
- [ Retrieving the Dataframes](#dataframes)<br>

[3. Data Preparation and Preprocessing](#preprocessing)<br>
- [3.1. Data Preparation](#preparation)<br>
  - [3.1.1. Applicants' Data](#applicantsprep)<br>
  - [3.1.2. Professional Activities' Data](#profactivprep)<br>
  - [3.1.3. Students' Data](#studentsprep)<br>
- [3.2. Missing Values](#missing)<br>
  - [3.2.1. Applicants' Data](#applicantsmissing)<br>
  - [3.2.2. Professional Activities' Data](#profactivmissing)<br>
  - [3.2.3. Students' Data](#studentsmissing)<br>
- [3.3. Removing Outliers](#outliers)<br>
- [3.4. Feature Engineering](#engineering)<br>
  - [3.4.1. Creating New Features](#newfeatures)<br>
  - [3.4.2. Encoding Categorical Variables](#encoding)<br>
  - [3.4.3. Splitting the Dataframes](#splitting)<br>
- [3.5. Storing the Dataframes](#storing)<br>

</div>

# <font color="#BFD62F">_____________</font>
# <font color='#5C666C'>Initial Setup</font> <a class="anchor" id="setup"></a>
[Back to Contents](#toc)

## <font color='#BFD62F'>Library and Functions Import</font> <a class="anchor" id="library"></a>
[Back to Contents](#toc)

In [1]:
#!python --version

We will be using the Python version 3.11.8.

In [2]:
#! pip install pandas==2.2.1
#! pip install numpy==1.24.4
#! pip install matplotlib==3.8.3
#! pip install seaborn==0.12.2
#! pip install plotly==5.20.0
#! pip install tenacity==8.2.2
#! pip install openpyxl>=3.1.0
#% pip install nbformat>=4.3.0
#! pip install rapidfuzz==3.11.0
#! pip install xlrd==2.0.1

In [3]:
%run Imports

In [4]:
import Functions as tf

## <font color='#BFD62F'>Retrieving the Dataframes </font> <a class="anchor" id="dataframes"></a>
[Back to Contents](#toc)

Let's retrieve the dataframes that we used in the previous notebook

In [5]:
%store -r df_applicants
%store -r df_profactiv
%store -r df_students

# <font color='#BFD62F'>_________________________________________</font>
# <font color='#5C666C'>3. Data Preparation and Preprocessing </font> <a class="anchor" id="preprocessing"></a>
[Back to Contents](#toc)

<br>

## <font color='#BFD62F'>3.1. Data Preparation</font> <a class="anchor" id="preparation"></a>
[Back to Contents](#toc)

We will start this stage of our work by performing some corrections and transformations that we identified to be important earlier in the exploration phase. We will do this for each of our datasets in separate. Nonetheless, we will maintain a copy of the original datasets, in case there is a need to resort to this version of the data in the future.

In [6]:
df_applicants_original = df_applicants.copy()
df_profactiv_original = df_profactiv.copy()
df_students_original = df_students.copy()

-------
### <font color='#BFD62F'>3.1.1. Applicants' Data </font> <a class="anchor" id="applicantsprep"></a>
[Back to Contents](#toc)

Starting with our applicants' data, we can drop the ID column that we found to not be neither unique nor useful to our work, `CD_CANDIDATO`. Recaping, we found this variable to have their unique values potentially associated with more than one applicant, with no relevant similarity between them.

In [7]:
df_applicants.drop(columns = {"CD_CANDIDATO"}, inplace = True)

Next, we can pivot the columns to transform the dataset into one where each row represent a certain application from a certain student, which is what we desire it to be.

Remember, we saw that each row was representing a past or current academic degree from a certain applicant performing a specific application. Therefore, we need to pivot the columns that refer to past or current academic degrees, creating new columns in the process. After this process, each row will have a unique `idCandidatura` (although `ID_INDIVIDUO` and `idcandidato` may appear more than once if the candidate has multiple applications).

In [8]:
df_applicants.head()

,ID_INDIVIDUO,idcandidato,idgenero,cd_pais,Localidade,DGrauAcademicoPT,DEstabEnsinoProvPT,DCursoProvPT,CursoConcluido,NotaOuMedia,DescEscala,nivelInglesCompreensao,nivelInglesFala,nivelInglesEscrita,idCandidatura,datacandidaturafim,dprograma,dcurso,DCursoOpcao,idEstadoCandidatura,DEstadoPT
0,23187.0,23011,M,IN,PUNE,Doutorado,Academia Militar,Master in Business Administration,S,7.44,grading system 1-10,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),25962,2022-03-11 10:27:35.000,Mestrado,Mestrado em Data Science and Advanced Analytics,especialização em Data Science,7,Não Admitido
1,27051.0,28982,M,SE,Vasteras,Doutorado,Ambrose Alli University,Administração e Contabilidade,S,2.2,Minímo positivo: 5 | Máximo: 10,Fluente (QERC: C2),Fluente (QERC: C2),Fluente (QERC: C2),33870,2023-05-16 17:26:12.750,Pós-Graduação,Pós-Graduação,Análise e Gestão de Risco,7,Não Admitido
2,27051.0,28982,M,SE,Vasteras,Doutorado,Ambrose Alli University,Administração e Contabilidade,S,2.2,Minímo positivo: 5 | Máximo: 10,Fluente (QERC: C2),Fluente (QERC: C2),Fluente (QERC: C2),33876,2023-05-17 07:13:49.393,Mestrado,Mestrado em Estatística e Gestão de Informação,especialização em Análise e Gestão de Informação,7,Não Admitido
3,26228.0,27230,M,PT,Lisbon,Doutorado,American International University-Bangladesh,MBA,S,Passed,CGPA- 3.35 out of 4.00,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),32150,2023-03-13 22:21:17.740,Master,Master degree program in Information Management,specialization in Business Intelligence - After Working Hours Format,7,Não Admitido
4,21700.0,20352,M,IR,Mashhad,Doutorado,Azad university,Ciências da Computação,N,18,Minimal positive: 10 | Maximum: 20,Fluente (QERC: C2),Fluente (QERC: C2),Avançado (QERC: C1),23479,2021-06-18 20:02:06.800,Master,Master degree program in Data Science and Advanced Analytics,specialization in Data Science,10,Desistiu após seleção


In [9]:
cols_to_pivot = ["DGrauAcademicoPT", "DEstabEnsinoProvPT", "DCursoProvPT", "CursoConcluido", "NotaOuMedia", "DescEscala"]

df_applicants = tf.pivot_columns(df_applicants, cols_to_pivot, "idCandidatura")
df_applicants.head()

,idCandidatura,DGrauAcademicoPT_1,DGrauAcademicoPT_2,DGrauAcademicoPT_3,DEstabEnsinoProvPT_1,DEstabEnsinoProvPT_2,DEstabEnsinoProvPT_3,DCursoProvPT_1,DCursoProvPT_2,DCursoProvPT_3,CursoConcluido_1,CursoConcluido_2,CursoConcluido_3,NotaOuMedia_1,NotaOuMedia_2,NotaOuMedia_3,DescEscala_1,DescEscala_2,DescEscala_3,ID_INDIVIDUO,idcandidato,idgenero,cd_pais,Localidade,nivelInglesCompreensao,nivelInglesFala,nivelInglesEscrita,datacandidaturafim,dprograma,dcurso,DCursoOpcao,idEstadoCandidatura,DEstadoPT
0,4344,Mestre,None,None,Universidade Federal de Lavras (Brasil),None,None,Estatística e Experimentação Agropecuária,None,None,S,None,None,2.86,None,None,Minimal positive: C | Maximum: A+,None,None,16108.0,730,M,MZ,Chimoio,Intermédio (QERC: B1-B2),Intermédio (QERC: B1-B2),Intermédio (QERC: B1-B2),2018-01-23 21:34:44.380,Pós-Graduação,Pós-Graduação,Sistemas de Informação Empresariais,4,Completa
1,6724,Licenciado,None,None,Coventry University,None,None,BA (Hons) Business Management,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,8855,M,CM,COVENTRY,NaN,NaN,NaN,2018-01-07 12:30:47.150,Post-Graduate,Postgraduate program,Enterprise Information Systems,4,Completa
2,6726,Licenciado,None,None,Anhanguera educacional,None,None,Engenharia de Produção Mecânica,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,8858,M,BR,taubate,NaN,NaN,NaN,2018-01-07 21:29:53.603,Pós-Graduação,Pós-Graduação,Sistemas de Informação Empresariais,4,Completa
3,6729,Sem Grau,None,None,Escola Secundaria Siumara Da Costa Primo,None,None,Ciências Sociais,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,8832,F,AO,Luanda,NaN,NaN,NaN,2018-01-08 11:30:37.810,Pós-Graduação,Pós-Graduação,Sistemas Estatísticos - Especialização em Estatísticas de Bancos Centrais,2,Em Análise
4,6731,Licenciado,None,None,Delhi university,None,None,BA Pass,None,None,S,None,None,45,None,None,Minimal positive: 35% | Maximum: 100%,None,None,NaN,8833,M,PT,amadora,Fluente (QERC: C2),Fluente (QERC: C2),Fluente (QERC: C2),2018-01-08 16:23:42.327,Pós-Graduação,Pós-Graduação,Sistemas de Informação Empresariais,4,Completa


Great! We can now check that `idCandidatura` is unique.

In [10]:
print("Number of rows in the dataset:", df_applicants.shape[0])
print("Number of unique values for 'idCandidatura':", df_applicants["idCandidatura"].nunique())

Number of rows in the dataset: 14946
Number of unique values for 'idCandidatura': 14946


Our last preparation (at this stage, at least) will be to disconsider from our dataset the candidates that are applying to programs that are not postgraduations or masters' degress. This is because the scope of our study is limited to these program types; in the future, our research can potentially be extended to accommodate these individuals.

In [11]:
df_applicants["dprograma"].unique()

array(['Pós-Graduação', 'Post-Graduate', 'Mestrado', 'Master',
       ' Postgraduate', 'Doutoramento', nan, 'PhD', "Bachelor's Degree ",
       'Certification Course', 'Licenciatura', 'Short Program',
       ' Pós-Graduação', 'Formação Avançada'], dtype=object)

In [12]:
programs_to_keep = ["Mestrado", "Pós-Graduação", "Master", " Postgraduate", "Post-Graduate", " Pós-Graduação"]

df_applicants = df_applicants[df_applicants["dprograma"].isin(programs_to_keep)]
print("Number of rows in the dataset after filtering program types:", df_applicants.shape[0])

Number of rows in the dataset after filtering program types: 14815


We only lost around 130 rows in the process, meaning that we kept more than 99.1% of our data.

Another important transformation that we should perform relates to the target, considering the high number of categories that we have at the moment. However, this distinction may be relevant further down the cleaning stage, prompting us to leave this change to the feature engineering phase.

Our last step before moving to the next table will be to ensured that our dataset is ordered by `datacandidaturafim`, so that the last candidates who applied are displayed in the last rows. This will come in handy when we perform the hold-out method.

In [13]:
df_applicants.sort_values(by = "datacandidaturafim", ascending = True, ignore_index = True, inplace = True)

-------
### <font color='#BFD62F'>3.1.2. Professional Activities' Data </font> <a class="anchor" id="profactivprep"></a>
[Back to Contents](#toc)

Moving on to the dataset that presents the past and current professional activities from the students, we will start by converting `DataFim` to the correct data type. For this, we will avail of the insights gathered during the exploration phase, and correct the one situation where the date was passed incorrectly.

In [14]:
df_profactiv["DataFim"] = pd.to_datetime(df_profactiv["DataFim"], errors = "coerce")

df_profactiv.loc[11449, "DataFim"] = pd.to_datetime("2019-07-01")

Nice.

We also learned that each row in this dataset represented a past or current professional experience from an applicant. Similarly to the previous dataset, this is not what we aim for each line to be. In this case, we want each row to be representative of a particular person, with the columns describing their past or current professional experiences.

In [15]:
df_profactiv.head()

,idCandidato,DataInicio,DataFim,ActivProEntidadePatronal,ActivProFuncao
0,182,2016-05-16,NaT,Linkare TI - Tecnologias de Informação,Business Analyst & Project Manager
1,182,2014-06-01,2016-05-15,Closer Consulting,Business Analyst & Project Manager
2,182,2010-04-01,2014-05-31,agap2IT Portugal,Business Analyst & Project Manager
3,286,2018-08-16,NaT,Escola Profissional de Ciências Geográficas,Técnica Especializada - Ensino de SIG
4,286,2018-02-01,NaT,Universidade Nova de Lisboa - Instituto Superior de Estatística e Gestão de Informação,Postgraduate Programs Manager


Before pivoting the columns, we should also reorder our dataset so that professional activities are always in chronological order. This is especially important since, in the exploration stage, we found that applicants did not follow this rule, and that was adding some confusion in the analysis of our data.

We will order the dataset by `idCandidato` and, within it, by `DataInicio` and then by `DataFim`.

In [16]:
df_profactiv.sort_values(by = ["idCandidato", "DataInicio", "DataFim"], ignore_index = True, inplace = True)

Our last step (before pivoting the columns) will be to disconsider being a student as a professional activity. Recaping our thoughts at the exploration stage, "". We will do this before pivoting since it is easier to directly remove these rows, but we will also check if the maximum number of activities taking by a single individual (given by the number of times the same `idCandidato` appears in the table) still holds.

In [17]:
df_profactiv = df_profactiv[~df_profactiv["ActivProFuncao"].isin(["Estudante", "Aluno", "Student"])]
df_profactiv["idCandidato"].value_counts().max()

6

In [18]:
cols_to_pivot = ["DataInicio", "DataFim", "ActivProEntidadePatronal", "ActivProFuncao"]

df_profactiv= tf.pivot_columns(df_profactiv, cols_to_pivot, "idCandidato", 6)

# Converting the "DataInicio_1" column to datetime, following what we explored in the exploration stage
df_profactiv["DataInicio_1"] = pd.to_datetime(df_profactiv["DataInicio_1"], errors = "coerce")
df_profactiv.head()

,idCandidato,DataInicio_1,DataInicio_2,DataInicio_3,DataInicio_4,DataInicio_5,DataInicio_6,DataFim_1,DataFim_2,DataFim_3,DataFim_4,DataFim_5,DataFim_6,ActivProEntidadePatronal_1,ActivProEntidadePatronal_2,ActivProEntidadePatronal_3,ActivProEntidadePatronal_4,ActivProEntidadePatronal_5,ActivProEntidadePatronal_6,ActivProFuncao_1,ActivProFuncao_2,ActivProFuncao_3,ActivProFuncao_4,ActivProFuncao_5,ActivProFuncao_6
0,121,2008-09-01,2018-09-01,2021-01-01,NaT,NaT,NaT,NaT,2020-12-31,NaT,NaT,NaT,NaT,"AEG Telecomunicações, SA",NaN,OutSystems,None,None,None,Responsável dos Sistemas de Informação,Customer Success Analytics Team Lead,Head Data & Analytics for R&D,None,None,None
1,182,2010-04-01,2014-06-01,2016-05-16,NaT,NaT,NaT,2014-05-31,2016-05-15,NaT,NaT,NaT,NaT,agap2IT Portugal,Closer Consulting,Linkare TI - Tecnologias de Informação,None,None,None,Business Analyst & Project Manager,Business Analyst & Project Manager,Business Analyst & Project Manager,None,None,None
2,286,2018-02-01,2018-08-16,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Universidade Nova de Lisboa - Instituto Superior de Estatística e Gestão de Informação,Escola Profissional de Ciências Geográficas,None,None,None,None,Postgraduate Programs Manager,Técnica Especializada - Ensino de SIG,None,None,None,None
3,296,2016-02-01,2019-07-19,2024-10-17,NaT,NaT,NaT,2019-07-19,2024-10-17,NaT,NaT,NaT,NaT,BNP Paribas,CISCO,Any,None,None,None,Client Support - Team Coordinator,Installed Base Analyst - Team Lead,Unemployed,None,None,None
4,366,2017-09-01,2018-02-01,NaT,NaT,NaT,NaT,2018-01-30,NaT,NaT,NaT,NaT,NaT,Nova-IMS,Nova-IMS,None,None,None,None,Professor Convidado de Ciência e Sistemas de Informação Geogáfica,Professor Convidado de Location Based Services,None,None,None,None


Let's also check that, after our transformations, each row refers to a unique `idCandidato`.

In [19]:
print("Number of rows in the dataset:", df_profactiv.shape[0])
print("Number of unique values for 'idCandidato':", df_profactiv["idCandidato"].nunique())

Number of rows in the dataset: 11485
Number of unique values for 'idCandidato': 11485


-------
### <font color='#BFD62F'>3.1.3. Students' Data </font> <a class="anchor" id="studentsprep"></a>
[Back to Contents](#toc)

For this dataset, we saw in the previous notebook that we have a relatively large number of students who are still yet to graduate, that is, students that do not have a value for our target variable, final GPA (`NotaFinal`). Therefore, we will filter them out, keeping only the individuals that are of relevance to our study. 

In [20]:
df_students = df_students[df_students["situacaoFinal"] == "Formado"]

# We will also be dropping the 2 students that are missing a value in the "NotaFinal" column
df_students.dropna(subset = ["NotaFinal"], inplace = True)
df_students.head()

,id_individuo,DT_MATRIC,dataConclusao,NotaFinal,situacaoFinal,cd_lectivo,cd_curso,nm_curso_pt,cd_discip,ds_discip_pt,notaFinalDisciplina
0,13686,2018-09-04,2020-07-07,17.0,Formado,201819,7512,Mestrado em Ciência de Dados e Métodos Analíticos Avançados,200142,Inteligência Computacional para Otimização,12.0
1,13686,2018-09-04,2020-07-07,17.0,Formado,201819,7512,Mestrado em Ciência de Dados e Métodos Analíticos Avançados,200144,Big Data,16.0
2,13686,2018-09-04,2020-07-07,17.0,Formado,201819,7512,Mestrado em Ciência de Dados e Métodos Analíticos Avançados,200174,Armazenamento e Recuperação de Dados,18.0
3,13686,2018-09-04,2020-07-07,17.0,Formado,201819,7512,Mestrado em Ciência de Dados e Métodos Analíticos Avançados,200175,Data Mining,15.0
4,13686,2018-09-04,2020-07-07,17.0,Formado,201819,7512,Mestrado em Ciência de Dados e Métodos Analíticos Avançados,200176,Visualização de Dados,16.0


Since we have no interest in analysing the grades for each individual course and we only identify the final GPA as being of relevance to our study, we have no need to keep more than one row for each student, and we can also drop the `cd_lectivo`, `cd_discip`, `ds_discip_pt`, and `notaFinalDisciplina` variables.

However, before doing this, there is one last transformation that we can perform. Earlier in the exploration stage, we created a variable that calculated the difference (in months) between the `dataConclusao` column of the df_students dataset, and the `datacandidaturafim` column of the df_applicants table. By doing that, we found that our merge of the two tables may have been incorrectly executed, and that our newly created variable may have also presented us with some incorrect values. Although this variable will not be used later on (since it provides us with information made available after the time of the application), we will keep it for now to evaluate if, this time, our merge of the datasets is better executed. We will also be keeping the earliest `DT_MATRIC` for the same purpose.

In [21]:
df_students.drop(columns = {"cd_lectivo", "cd_discip", "ds_discip_pt", "notaFinalDisciplina"}, inplace = True)

In [22]:
df_students["DT_MATRIC"] = df_students.groupby("id_individuo")["DT_MATRIC"].transform("min")
df_students = df_students.groupby(["id_individuo", "nm_curso_pt"], as_index = False).last()

# Coherence check to ensure no student graduated before their enrollment
df_students[df_students["DT_MATRIC"] > df_students["dataConclusao"]]

,id_individuo,nm_curso_pt,DT_MATRIC,dataConclusao,NotaFinal,situacaoFinal,cd_curso


In [23]:
df_students.head()

,id_individuo,nm_curso_pt,DT_MATRIC,dataConclusao,NotaFinal,situacaoFinal,cd_curso
0,13686,Mestrado em Ciência de Dados e Métodos Analíticos Avançados,2018-09-04,2020-07-07,17.0,Formado,7512
1,13691,Mestrado em Ciência de Dados e Métodos Analíticos Avançados,2018-09-05,2021-11-26,13.0,Formado,7512
2,13694,Mestrado em Ciência de Dados e Métodos Analíticos Avançados,2018-09-04,2021-01-25,17.0,Formado,7512
3,13696,Mestrado em Gestão de Informação,2018-09-05,2020-02-27,15.0,Formado,9434
4,13702,Mestrado em Gestão de Informação,2018-03-23,2022-01-26,14.0,Formado,9434


## <font color='#BFD62F'>3.2. Missing Values</font> <a class="anchor" id="missing"></a>
[Back to Contents](#toc)

Let's now handle the missing values we identified earlier in the exploration notebook. While some may eventually be "replaceable", others may have a plausible justification for their presence (in the context of the problem) or may not be realistic to replace.

Once again, we will perform this one dataframe at a time. It is likely that new missing values arise when we merge the three datasets into one, but we will explore these cases when we get there.

-------
### <font color='#BFD62F'>3.2.1. Applicants' Data </font> <a class="anchor" id="applicantsmissing"></a>
[Back to Contents](#toc)

In [24]:
df_applicants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14815 entries, 0 to 14814
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   idCandidatura           14815 non-null  int64         
 1   DGrauAcademicoPT_1      14815 non-null  object        
 2   DGrauAcademicoPT_2      4282 non-null   object        
 3   DGrauAcademicoPT_3      923 non-null    object        
 4   DEstabEnsinoProvPT_1    14755 non-null  object        
 5   DEstabEnsinoProvPT_2    4268 non-null   object        
 6   DEstabEnsinoProvPT_3    916 non-null    object        
 7   DCursoProvPT_1          14750 non-null  object        
 8   DCursoProvPT_2          4267 non-null   object        
 9   DCursoProvPT_3          911 non-null    object        
 10  CursoConcluido_1        14797 non-null  object        
 11  CursoConcluido_2        4282 non-null   object        
 12  CursoConcluido_3        923 non-null    object

We have missing values in the following variables:
* `DGrauAcademicoPT_2` and `DGrauAcademicoPT_3` - The most obvious justification for this is that the applicants did not take a second and/or third academic degree. Therefore, these missing values should not be filled.
* `DEstabEnsinoProvPT_1`, `DEstabEnsinoProvPT_2` and `DEstabEnsinoProvPT_3` - Filling these missing values (in the cases where the plausible justification of not having the degrees does not hold) is an almost impossible task, as we are dealing with a subset with a great number of unique values. Filling it with the mode (which would, for the first and third variables, be Nova IMS) would likely give us incorrect information that could severely hamper our predictive abilities (also considering that being a previous Nova IMS student can be an important predictor of admission and performance).
* `DCursoProvPT_1`, `DCursoProvPT_2` and `DCursoProvPT_3` - A similar justification to the one given above can be provided for not filling the missing values in these variables.
* `CursoConcluido_1`, `CursoConcluido_2` and `CursoConcluido_3` - A similar justification to the one given above can be provided for not filling the missing values in these variables.
* `NotaOuMedia_1`, `NotaOuMedia_2`, `NotaOuMedia_3`, `DescEscala_1`, `DescEscala_2` and `DescEscala_3` - In the current state of these variables, filling these missing values would have the same complexity as the variables mentioned above. When we transform these attributes (something we intend to do in the feature engineering phase), perhaps we can explore the possibility of filling missing values where appropriate.
* `ID_INDIVIDUO` - Being an ID column, filling missing values would not be recommended. However, since we know it to be directly associated with `idcandidato`, we can try to do it by relating the two attributes.
* `cd_pais` and `Localidade` - We have a very reduced number of missing values, but we can try to fill them by exploring other variables in our dataset.
* `nivelInglesCompreensao`, `nivelInglesFala` and `nivelInglesEscrita` - We can assess the possibility of accurately filling the missing values by exploring the values for the other variables in our dataset. Where it is not possible, we are likely to fill them with the mode (since the number of missing values is not very subtantial).
* `dcurso` and `DCursoOpcao` - The number of missing values is especially reduced in these cases, but we can explore the values of the other related variables in order to fill them.

Adding to the "explicitly-mentioned" missing values, the exploration stage showed us some values that, despite being defined, may mask others in our data. This may be happening, for example, in the `DGrauAcademicoPT_1` and `idgenero` variables.

#### ID_INDIVIDUO

Our first idea will be to replace the missing values in this column with respect to their values in the `idcandidato` column. For instance, if there are 2 rows with the same `idcandidato`, and `ID_INDIVIDUO` is missing in one of them, then the missing value should be filled by the `ID_INDIVIDUO` value of the row with the same `idcandidato`.

In [25]:
print("Number of missing values before the transformation:", df_applicants["ID_INDIVIDUO"].isna().sum())
df_applicants["ID_INDIVIDUO"] = df_applicants["ID_INDIVIDUO"].fillna(df_applicants.groupby("idcandidato")["ID_INDIVIDUO"].transform("first"))
print("Number of missing values after the transformation:", df_applicants["ID_INDIVIDUO"].isna().sum())

Number of missing values before the transformation: 23
Number of missing values after the transformation: 23


Unfortunately, our strategy didn't yield any benefit and the number of missing values remained the same. Although we could simply delete these rows, we can keep them for the task of predicting admissions, since this column will be of no relevance (since it is an ID column) and will be deleted. When it comes to predicting grades, this variable would be important to allow us the merge the df_applicants with the df_students datasets. Therefore, these observations will most likely be disconsidered.

#### cd_pais and Localidade

Let's check the rows where these values are missing.

In [26]:
df_applicants[df_applicants["cd_pais"].isna() | df_applicants["Localidade"].isna()]

,idCandidatura,DGrauAcademicoPT_1,DGrauAcademicoPT_2,DGrauAcademicoPT_3,DEstabEnsinoProvPT_1,DEstabEnsinoProvPT_2,DEstabEnsinoProvPT_3,DCursoProvPT_1,DCursoProvPT_2,DCursoProvPT_3,CursoConcluido_1,CursoConcluido_2,CursoConcluido_3,NotaOuMedia_1,NotaOuMedia_2,NotaOuMedia_3,DescEscala_1,DescEscala_2,DescEscala_3,ID_INDIVIDUO,idcandidato,idgenero,cd_pais,Localidade,nivelInglesCompreensao,nivelInglesFala,nivelInglesEscrita,datacandidaturafim,dprograma,dcurso,DCursoOpcao,idEstadoCandidatura,DEstadoPT
1202,9897,Licenciado,None,None,"CATHOLIC UNIVERSITY OF CAMEROON, BAMENDA.",None,None,Microbiologia,None,None,S,None,None,2.96,None,None,"Minimal positive: 0,5 | Maximum: 4",None,None,15038.0,10754,F,CM,NaN,Fluente (QERC: C2),Fluente (QERC: C2),Fluente (QERC: C2),2018-07-04 17:30:01.253,Mestrado,Mestrado em Estatística e Gestão de Informação,Especialização em Análise e Gestão de Informação,7,Não Admitido
2073,11294,Licenciado,None,None,International University of Management,None,None,business information and systems,None,None,S,None,None,61,None,None,minimo positivo 50 maximo positivo 100,None,None,16575.0,11492,M,NaN,lisboa,Fluente (QERC: C2),Fluente (QERC: C2),Fluente (QERC: C2),2019-03-13 19:20:52.433,Mestrado,Mestrado em Gestão de Informação,Especialização em Gestão de Sistemas e Tecnologias de Informação – Diurno,7,Não Admitido
5685,20343,Licenciado,None,None,University Of Cape Town,None,None,Bachelors of Commerce specializing in Information Systems,None,None,S,None,None,72.92,None,None,Minimal positive: 50% | Maximum: 100%,None,None,20540.0,18377,M,NaN,Windhoek,Nativo ou bilingue,Nativo ou bilingue,Nativo ou bilingue,2021-02-05 10:30:48.773,Master,Master degree program in Information Management,specialization in Information Systems and Technologies Management - Working Hours Format,6,Admitido
7972,25707,Licenciado,None,None,Namibia University of Science and Technology,None,None,BACHELOR OF BUSINESS ADMINISTRATION,None,None,S,None,None,B,None,None,Minimal positive: C | Maximum: A,None,None,23083.0,22806,F,NaN,Namibia,Fluente (QERC: C2),Fluente (QERC: C2),Fluente (QERC: C2),2022-03-04 06:17:50.363,Master,Master degree program in Data Science and Advanced Analytics,specialization in Business Analytics,10,Desistiu após seleção
8191,26071,Licenciado,None,None,IADE- Creative University,None,None,Marketing e Publicidade,None,None,S,None,None,15,None,None,Minímo positivo: 10 | Máximo: 20,None,None,23233.0,21440,F,NaN,NaN,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),2022-03-14 12:25:49.993,Mestrado,Mestrado em Data Driven Marketing,especialização em Digital Marketing and Analytics - Regime Pós-Laboral,6,Admitido


We can fill these missing values in a case-by-case scenario. The only situation where both `cd_pais` and `Localidade` are missing refers to an applicant that took their bachelor's degree in a HEI in Lisbon, Portugal. Therefore, we will assume it to be the applicant's location and country of origin, respectively.

In [27]:
# Checking that each of these applicants only appears once in the datset
#df_applicants[df_applicants["idcandidato"] == 10754]
#df_applicants[df_applicants["idcandidato"] == 11492]
#df_applicants[df_applicants["idcandidato"] == 18377]
#df_applicants[df_applicants["idcandidato"] == 22806]
#df_applicants[df_applicants["idcandidato"] == 21440]

In [28]:
df_applicants.loc[df_applicants["idcandidato"] == 10754, "Localidade"] = "Bamenda"
df_applicants.loc[df_applicants["idcandidato"] == 11492, "cd_pais"] = "PT"
df_applicants.loc[df_applicants["idcandidato"] == 18377, "cd_pais"] = "NA"
df_applicants.loc[df_applicants["idcandidato"] == 22806, "cd_pais"] = "NA"
df_applicants.loc[df_applicants["idcandidato"] == 21440, "cd_pais"] = "PT"
df_applicants.loc[df_applicants["idcandidato"] == 21440, "Localidade"] = "Lisboa"

print("Number of missing values after manual corrections:", len(df_applicants[df_applicants["cd_pais"].isna() |
                                                                              df_applicants["Localidade"].isna()]))

Number of missing values after manual corrections: 0


#### nivelInglesCompreensao, nivelInglesFala and nivelInglesEscrita

Once again, let's view the rows where the missing value occur.

In [29]:
df_applicants[df_applicants["nivelInglesCompreensao"].isna() |
              df_applicants["nivelInglesFala"].isna() |
              df_applicants["nivelInglesEscrita"].isna()]

,idCandidatura,DGrauAcademicoPT_1,DGrauAcademicoPT_2,DGrauAcademicoPT_3,DEstabEnsinoProvPT_1,DEstabEnsinoProvPT_2,DEstabEnsinoProvPT_3,DCursoProvPT_1,DCursoProvPT_2,DCursoProvPT_3,CursoConcluido_1,CursoConcluido_2,CursoConcluido_3,NotaOuMedia_1,NotaOuMedia_2,NotaOuMedia_3,DescEscala_1,DescEscala_2,DescEscala_3,ID_INDIVIDUO,idcandidato,idgenero,cd_pais,Localidade,nivelInglesCompreensao,nivelInglesFala,nivelInglesEscrita,datacandidaturafim,dprograma,dcurso,DCursoOpcao,idEstadoCandidatura,DEstadoPT
0,6724,Licenciado,None,None,Coventry University,None,None,BA (Hons) Business Management,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,8855,M,CM,COVENTRY,NaN,NaN,NaN,2018-01-07 12:30:47.150,Post-Graduate,Postgraduate program,Enterprise Information Systems,4,Completa
1,6726,Licenciado,None,None,Anhanguera educacional,None,None,Engenharia de Produção Mecânica,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,8858,M,BR,taubate,NaN,NaN,NaN,2018-01-07 21:29:53.603,Pós-Graduação,Pós-Graduação,Sistemas de Informação Empresariais,4,Completa
2,6729,Sem Grau,None,None,Escola Secundaria Siumara Da Costa Primo,None,None,Ciências Sociais,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,8832,F,AO,Luanda,NaN,NaN,NaN,2018-01-08 11:30:37.810,Pós-Graduação,Pós-Graduação,Sistemas Estatísticos - Especialização em Estatísticas de Bancos Centrais,2,Em Análise
4,6737,Licenciado,None,None,UAN,None,None,Engenharia da Gestão e Ordenamento,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,7016,M,AO,Luanda,NaN,NaN,NaN,2018-01-09 16:00:56.433,Pós-Graduação,Pós-Graduação,Inteligência Geoespacial,4,Completa
5,6738,Licenciado,None,None,Universidade do Minho,None,None,Sistemas de Informação,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,8876,M,PT,Lisboa,NaN,NaN,NaN,2018-01-09 21:30:43.620,Pós-Graduação,Pós-Graduação,Sistemas de Informação Empresariais,6,Admitido
8,6744,Licenciado,None,None,Universidade Estadual Paulista - UNESP,None,None,Gestão,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,8720,F,BR,Bauru,NaN,NaN,NaN,2018-01-10 22:52:27.310,Pós-Graduação,Pós-Graduação,Sistemas de Informação Empresariais,6,Admitido
9,6745,Licenciado,None,None,AIEC - ASSOCIAÇÃO INTERNACIONAL DE ENSINO CONTINUADO,None,None,Administração e Gestão de Empresas,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,5545,F,BR,JABOATAO GUARARAPES,NaN,NaN,NaN,2018-01-11 10:19:09.530,Pós-Graduação,Pós-Graduação,Sistemas de Informação Empresariais,6,Admitido
10,6746,Licenciado,None,None,Universidade Aberta,None,None,Ciências Sociais,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,8888,M,PT,caneças,NaN,NaN,NaN,2018-01-11 11:37:32.260,Pós-Graduação,Pós-Graduação,Gestão de Informações e Segurança,6,Admitido
11,6749,Licenciado,None,None,Instituto Politécnico de Leiria - Escola Superior de Tecnologia e Gestão de Leiria,None,None,Engenharia Informática,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,8893,M,PT,Olival,NaN,NaN,NaN,2018-01-12 14:17:36.043,Pós-Graduação,Pós-Graduação,Sistemas de Informação Empresariais,6,Admitido
13,6768,Licenciado,None,None,Escola Universitária Vasco da Gama,None,None,Aconselhamento Psicossocial,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,8919,M,SA,Jeddah,NaN,NaN,NaN,2018-01-16 07:06:51.247,Post-Graduate,Postgraduate program,Enterprise Information Systems,6,Admitido


The number of rows is not significant. Since there is no way of undoubtebly replacing the missing values for these variables, we will fill them with their respective modes (which we already saw in the exploration phase but will be displayed again the cell below).

In [30]:
print("Mode for 'nivelInglesCompreensao':", df_applicants["nivelInglesCompreensao"].mode()[0])
print("Mode for 'nivelInglesFala':", df_applicants["nivelInglesFala"].mode()[0])
print("Mode for 'nivelInglesEscrita':", df_applicants["nivelInglesEscrita"].mode()[0])

Mode for 'nivelInglesCompreensao': Avançado (QERC: C1)
Mode for 'nivelInglesFala': Avançado (QERC: C1)
Mode for 'nivelInglesEscrita': Avançado (QERC: C1)


In [31]:
df_applicants["nivelInglesCompreensao"].fillna(df_applicants["nivelInglesCompreensao"].mode()[0], inplace = True)
df_applicants["nivelInglesFala"].fillna(df_applicants["nivelInglesFala"].mode()[0], inplace = True)
df_applicants["nivelInglesEscrita"].fillna(df_applicants["nivelInglesEscrita"].mode()[0], inplace = True)

In [32]:
print("Number of missing values after the transformations:", len(df_applicants[df_applicants["nivelInglesCompreensao"].isna() |
                                                                               df_applicants["nivelInglesFala"].isna() | 
                                                                               df_applicants["nivelInglesEscrita"].isna()]))

Number of missing values after the transformations: 0


#### dcurso and DCursoOpcao

In [33]:
df_applicants[df_applicants["dcurso"].isna() | df_applicants["DCursoOpcao"].isna()]

,idCandidatura,DGrauAcademicoPT_1,DGrauAcademicoPT_2,DGrauAcademicoPT_3,DEstabEnsinoProvPT_1,DEstabEnsinoProvPT_2,DEstabEnsinoProvPT_3,DCursoProvPT_1,DCursoProvPT_2,DCursoProvPT_3,CursoConcluido_1,CursoConcluido_2,CursoConcluido_3,NotaOuMedia_1,NotaOuMedia_2,NotaOuMedia_3,DescEscala_1,DescEscala_2,DescEscala_3,ID_INDIVIDUO,idcandidato,idgenero,cd_pais,Localidade,nivelInglesCompreensao,nivelInglesFala,nivelInglesEscrita,datacandidaturafim,dprograma,dcurso,DCursoOpcao,idEstadoCandidatura,DEstadoPT
1944,11562,Licenciado,None,None,Università degli Studi Guglielmo Marconi,None,None,Economic Science,None,None,F,None,None,96,None,None,Minimal positive: 66-69 | Maximum: 110,None,None,16414.0,12412,M,IT,Alassio,Avançado (QERC: C1),Intermédio (QERC: B1-B2),Avançado (QERC: C1),2019-03-11 18:55:34.360,Postgraduate,Postgraduate program,NaN,10,Desistiu após seleção
2020,11973,Licenciado,Mestre,Sem Grau,"Faculty of Management and Information Technology, State Polytechnical University St Petersburg","Faculty of Economics, University of Ljubljana",Universidade Nova de Lisboa,Public and Municipal Management,Master in International Business,Nova IMS Data Science for Marketing,S,S,N,"4,70","9,50",17,Minimal positive: 0 | Maximum: 5,Minimal positive: 5 | Maximum: 10,Minimal positive: 10 | Maximum: 20,16487.0,12673,F,SI,Ljubljana,Fluente (QERC: C2),Fluente (QERC: C2),Fluente (QERC: C2),2019-03-13 11:26:20.813,Postgraduate,Postgraduate program,NaN,6,Admitido
4257,16664,Licenciado,None,None,Universidade de Lisboa,None,None,Management,None,None,S,None,None,13,None,None,Minimal positive: 10 | Maximum: 20,None,None,18770.0,15519,F,PT,Lisboa,Avançado (QERC: C1),Intermédio (QERC: B1-B2),Intermédio (QERC: B1-B2),2020-03-13 11:30:37.720,Master,NaN,NaN,6,Admitido
4757,17480,Licenciado,None,None,"Univeristy of Ilorin, Ilorin, Nigeria.",None,None,Finanças,None,None,S,None,None,2.2,None,None,Minimal positive: 0 | Maximum: 5,None,None,19353.0,16084,F,NG,Ogun State.,Nativo ou bilingue,Nativo ou bilingue,Nativo ou bilingue,2020-05-18 22:11:50.250,Master,NaN,NaN,10,Desistiu após seleção


We can check if the applicants that were admitted appear in the df_students dataset. If so, the name of the program should be displayed there.

In [34]:
# Checking that each of these applicants only appears once in the datset
#df_applicants[df_applicants["idcandidato"] == 12673]  # Appears twice in the data, but in the other the applicant declined the enrollment offer.
#df_applicants[df_applicants["idcandidato"] == 15519]

In [35]:
df_students[df_students["id_individuo"] == 16487]

,id_individuo,nm_curso_pt,DT_MATRIC,dataConclusao,NotaFinal,situacaoFinal,cd_curso
793,16487,Pós-Graduação em Data Science for Marketing,2019-03-26,2020-06-26,17.0,Formado,4977


In [36]:
df_students[df_students["id_individuo"] == 18770]

,id_individuo,nm_curso_pt,DT_MATRIC,dataConclusao,NotaFinal,situacaoFinal,cd_curso


We can fill the missing value for the `id_individuo` 16487. For the remaining cases, we will opt to drop the observations, since the cost of trying to fill them with a specific value (such as a mode) may be larger than the benefit of keeping these rows.

In [37]:
df_applicants.loc[df_applicants["ID_INDIVIDUO"] == 16487, "DCursoOpcao"] = "Pós-Graduação em Data Science for Marketing"

df_applicants.dropna(subset = ["dcurso", "DCursoOpcao"], inplace = True)

print("Number of missing values after the transformations:", len(df_applicants[df_applicants["dcurso"].isna() |
                                                                               df_applicants["DCursoOpcao"].isna()]))

Number of missing values after the transformations: 0


#### DGrauAcademicoPT_1

Earlier, we identified a small number of observations whose first academic experience gave them no degree ("Sem Grau"). Since this is not very common, we can dig deeper into these cases in order to assess if they can be deemed as missing values.

In [38]:
df_applicants[df_applicants["DGrauAcademicoPT_1"] == "Sem Grau"]

,idCandidatura,DGrauAcademicoPT_1,DGrauAcademicoPT_2,DGrauAcademicoPT_3,DEstabEnsinoProvPT_1,DEstabEnsinoProvPT_2,DEstabEnsinoProvPT_3,DCursoProvPT_1,DCursoProvPT_2,DCursoProvPT_3,CursoConcluido_1,CursoConcluido_2,CursoConcluido_3,NotaOuMedia_1,NotaOuMedia_2,NotaOuMedia_3,DescEscala_1,DescEscala_2,DescEscala_3,ID_INDIVIDUO,idcandidato,idgenero,cd_pais,Localidade,nivelInglesCompreensao,nivelInglesFala,nivelInglesEscrita,datacandidaturafim,dprograma,dcurso,DCursoOpcao,idEstadoCandidatura,DEstadoPT
2,6729,Sem Grau,None,None,Escola Secundaria Siumara Da Costa Primo,None,None,Ciências Sociais,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,8832,F,AO,Luanda,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),2018-01-08 11:30:37.810,Pós-Graduação,Pós-Graduação,Sistemas Estatísticos - Especialização em Estatísticas de Bancos Centrais,2,Em Análise
588,8428,Sem Grau,None,None,Instituto Politécnico de Setúbal - Escola Superior de Tecnologia de Setúbal,None,None,"Instalações Eléctricas, Manutenção e Automação",None,None,S,None,None,14,None,None,Minímo positivo: 10 | Máximo: 20,None,None,14359.0,9931,M,PT,Moita,Fluente (QERC: C2),Intermédio (QERC: B1-B2),Intermédio (QERC: B1-B2),2018-04-20 11:30:03.067,Pós-Graduação,Pós-Graduação,Gestão de Informações e Segurança,6,Admitido
2339,12204,Sem Grau,None,None,12º ano,None,None,Ciências Económicas,None,None,S,None,None,13,None,None,Minímo positivo: 10 | Máximo: 20,None,None,16459.0,12778,M,PT,Lisboa,Intermédio (QERC: B1-B2),Intermédio (QERC: B1-B2),Intermédio (QERC: B1-B2),2019-03-18 10:59:20.940,Pós-Graduação,Pós-Graduação,Marketing Research e CRM,7,Não Admitido
3955,16741,Sem Grau,None,None,University of buea,None,None,SMS GEOGRAPHY,None,None,S,None,None,GPA,None,None,2.84,None,None,18685.0,16038,F,CM,Buea,Fluente (QERC: C2),Fluente (QERC: C2),Fluente (QERC: C2),2020-03-11 08:32:07.693,Pós-Graduação,Pós-Graduação,Business Analytics for Hospitality & Tourism,7,Não Admitido
4308,16751,Sem Grau,None,None,Escola Secundária Quinta do Marquês,None,None,Tecnológico de Informática,None,None,F,None,None,11,None,None,Minímo positivo: 10 | Máximo: 20,None,None,19019.0,15921,M,PT,Estoril,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),2020-03-16 17:30:15.863,Pós-Graduação,Pós-Graduação,Gestão dos Sistemas e Tecnologias de Informação,7,Não Admitido
4391,16503,Sem Grau,Sem Grau,None,"IADE-U, Instituto de Arte, Design e Empresa",Universidade Lusófona de Humanidades e Tecnologias,None,Design,Comunicação e Jornalismo,None,N,N,None,17,15,None,Minímo positivo: 10 | Máximo: 20,Minímo positivo: 10 | Máximo: 20,None,18556.0,15043,F,PT,Bobadela,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),2020-04-07 14:54:57.887,Pós-Graduação,Pós-Graduação,Digital Marketing and Analytics,6,Admitido
5386,19549,Sem Grau,None,None,Instituto Politécnico de Lisboa - Instituto Superior de Engenharia de Lisboa,None,None,Electrónica e Telecomunicações,None,None,N,None,None,12,None,None,Minímo positivo: 10 | Máximo: 20,None,None,20188.0,18038,M,PT,Estoril,Fluente (QERC: C2),Fluente (QERC: C2),Fluente (QERC: C2),2020-11-19 10:47:14.613,Pós-Graduação,Pós-Graduação,Digital Enterprise Management,6,Admitido
5472,19835,Sem Grau,None,None,Universidade Técnica de Lisboa - Instituto Superior de Economia e Gestão,None,None,Economia,None,None,S,None,None,14,None,None,Minímo positivo: 10 | Máximo: 20,None,None,20286.0,18404,M,PT,Vila Nova da Barquinha,Intermédio (QERC: B1-B2),Intermédio (QERC: B1-B2),Intermédio (QERC: B1-B2),2021-01-20 09:38:11.660,Pós-Graduação,Pós-Graduação,Gestão e Controlo Financeiro e Orçamental,6,Admitido
5566,20109,Sem Grau,None,None,Universidade do Minho,None,None,Gestão,None,None,F,None,None,16,None,None,Minímo positivo: 10 | Máximo: 20,None,None,20429.0,18550,M,PT,Braga,Fluente (QERC: C2),Fluente (QERC: C2),Avançado (QERC: C1),2021-01-25 18:27:36.840,Pós-Graduação,Pós-Graduação,Data Science for Finance,10,Desistiu após seleção
5567,20115,Sem Grau,None,None,Universidade do Minho,None,No

Some of these cases represent students who took a bachelor's, while others refer to students that attended higher education but did not necessarily a received a bachelor's degree (such as "Politécnico" students), and others even mention their secondary school as their first educational experience.

Although we could manually correct these observations in a case-by-case fashion, this could compromise the scalability of our solution, since this would not be feasible if we were to analyse every single applicant's academic background before passing it to the model. Therefore, for the time being, __we will accept this noise in our data__, hoping that our models can see through it and accurately predict admission and performance for these applicants.

#### idgenero

Regarding our gender variable, `idgenero`, we identified some situations where, although the value was not explicitly mentioned to be missing, its content was empty (as shown below).

In [39]:
df_applicants["idgenero"].value_counts()

idgenero
M    8384
F    6315
      113
Name: count, dtype: int64

Our initial assumption (and the one that will likely hold) is that these observations refer to students that do not identify with neither of the two most represented genders. However, before proceeding with this justification, we should check if there are other rows for the same applicant where this column does not show an empty value.

In [40]:
applicants_with_empty_value = df_applicants.loc[df_applicants["idgenero"] == " ", "idcandidato"]

applicants_with_empty_and_male_or_female = df_applicants[(df_applicants["idcandidato"].isin(applicants_with_empty_value)) &
                                                         (df_applicants["idgenero"].isin(["M", "F"]))]["idcandidato"]
df_applicants[(df_applicants["idcandidato"].isin(applicants_with_empty_and_male_or_female)) & (df_applicants["idgenero"] == " ")]

,idCandidatura,DGrauAcademicoPT_1,DGrauAcademicoPT_2,DGrauAcademicoPT_3,DEstabEnsinoProvPT_1,DEstabEnsinoProvPT_2,DEstabEnsinoProvPT_3,DCursoProvPT_1,DCursoProvPT_2,DCursoProvPT_3,CursoConcluido_1,CursoConcluido_2,CursoConcluido_3,NotaOuMedia_1,NotaOuMedia_2,NotaOuMedia_3,DescEscala_1,DescEscala_2,DescEscala_3,ID_INDIVIDUO,idcandidato,idgenero,cd_pais,Localidade,nivelInglesCompreensao,nivelInglesFala,nivelInglesEscrita,datacandidaturafim,dprograma,dcurso,DCursoOpcao,idEstadoCandidatura,DEstadoPT


None of the applicants whose value for `idgenero` is empty is repeated throughout the dataset with one of the two main genders. Consequently, we will proceed with our initial theory.

For clarity purposes, we will simply replace these empty values with "Other".

In [41]:
print("Number of applicants that do not identify with neither of the two main genders:",
      df_applicants[df_applicants["idgenero"] == " "]["idcandidato"].nunique())

Number of applicants that do not identify with neither of the two main genders: 102


In [42]:
df_applicants["idgenero"] = df_applicants["idgenero"].str.replace(" ", "Other")
df_applicants["idgenero"].value_counts()

idgenero
M        8384
F        6315
Other     113
Name: count, dtype: int64

For the time being, these are the missing values that will be handled in our df_applicants dataset. Let's now move on to our table with the data about the applicants' past and current professional experiences.

-------
### <font color='#BFD62F'>3.2.2. Professional Activities' Data </font> <a class="anchor" id="profactivmissing"></a>
[Back to Contents](#toc)


In [43]:
df_profactiv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11485 entries, 0 to 11484
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   idCandidato                 11485 non-null  int64         
 1   DataInicio_1                10565 non-null  datetime64[ns]
 2   DataInicio_2                6012 non-null   datetime64[ns]
 3   DataInicio_3                2968 non-null   datetime64[ns]
 4   DataInicio_4                15 non-null     datetime64[ns]
 5   DataInicio_5                4 non-null      datetime64[ns]
 6   DataInicio_6                2 non-null      datetime64[ns]
 7   DataFim_1                   5611 non-null   datetime64[ns]
 8   DataFim_2                   3129 non-null   datetime64[ns]
 9   DataFim_3                   335 non-null    datetime64[ns]
 10  DataFim_4                   7 non-null      datetime64[ns]
 11  DataFim_5                   2 non-null      datetime64

There are several missing values in almost all columns of the df_profactiv table (the exception is our ID variable for this dataset, `idCandidato`). While some of these values may be justifiable (since not all applicants have 6 past or current professional experiences), others may simply have been omitted by the candidates.

Given the high number of unique values for the `ActivProEntidadePatronal` and `ActivProFuncao` subsets, filling the missing ones would result in the addition of too much noise, which we naturally want to avoid. For the date variables (`DataInicio` and `DataFim`), we can explore if it could be reasonable to fill the missing values of `DataFim` with the `DataInicio` of the following experience, and the missing values of `DataInicio` with the `DataFim` of the the past experience, but we have to explore if this regularly happens in other observations whose values are not missing.

In [44]:
df_profactiv[~df_profactiv["DataInicio_4"].isna()]

,idCandidato,DataInicio_1,DataInicio_2,DataInicio_3,DataInicio_4,DataInicio_5,DataInicio_6,DataFim_1,DataFim_2,DataFim_3,DataFim_4,DataFim_5,DataFim_6,ActivProEntidadePatronal_1,ActivProEntidadePatronal_2,ActivProEntidadePatronal_3,ActivProEntidadePatronal_4,ActivProEntidadePatronal_5,ActivProEntidadePatronal_6,ActivProFuncao_1,ActivProFuncao_2,ActivProFuncao_3,ActivProFuncao_4,ActivProFuncao_5,ActivProFuncao_6
2017,12499,2007-08-07,2010-01-25,2016-09-30,2017-08-07,2019-09-23,NaT,2019-03-31,2016-08-30,2007-08-06,2019-09-20,NaT,NaT,NOVA School of Business & Economics,SIEMENS,HUAWEI Technologies,NOVA SBE,NOS SGPS,NaN,Head Of Business Intelligence & Quality Assurance,Business Controller,Business Controller,Head of School Data Analytics and quality assurance,Head Of Financial Control and asset management,NaN
2616,13778,2008-11-03,2013-01-07,2013-01-07,2016-06-06,NaT,NaT,2016-06-03,2014-07-14,2014-07-14,NaT,NaT,NaT,"SPAES, Lda",Galp Energia / Frauenthal Automotive,Galp Energia / Frauenthal Automotive,"Impersol, Lda",None,None,Direcção Dept. Gestão de Projectos,Estagiário Bolseiro,Estágiário Bolseiro,Direcção Dept. Técnico,None,None
2692,13979,2011-11-07,2014-03-24,2014-03-24,2019-05-11,NaT,NaT,2011-01-11,2019-07-02,NaT,2019-07-02,NaT,NaT,Philip Morris,Banco do Brasil (funcionária pública),Banco do Brasil (funcionária pública),Preciso Viajar (blog de viagens),None,None,Analista de Produto Sr.,Assistente de Negócios,Assistente de Negócios,Empreendedora - Blogueira,None,None
2767,14226,2014-01-06,2017-09-03,2017-09-03,2017-09-03,2018-09-17,2018-09-17,2017-09-01,2018-08-31,2018-08-31,2018-08-31,2018-08-31,NaT,Henkel,Henkel,Ledvance,Henkel,Henkel,Ledvance,Account Manager,Account Manager,Key Account Manager Junior,Key Account Manager Junior,Key Account Manager Junior,Key Account Manager Junior
2910,14553,2018-01-08,2018-01-08,2018-11-09,2020-03-09,NaT,NaT,2018-11-08,2018-11-08,2020-03-08,NaT,NaT,NaT,BNP Paribas Corporate and Institutional Banking,BNP Paribas,"Banco Invest, S.A.",OmiClear - The Iberian Energy Clearing House,None,None,Cross-asset regulatory reporting officer,Cross-asset regulatory reporting officer,Quantitative Risk Analyst,Analyst,None,None
3022,14769,2018-02-12,2018-08-01,2019-01-01,2019-02-11,NaT,NaT,2018-08-01,2019-01-31,NaT,2019-12-31,NaT,NaT,Capgemini,Capgemini,Ultra-Controlo,Johnson&Johnson,None,None,Trainee,Junior Consultant,Technical Sales Engineer,Clinical Support Trainee,None,None
3499,15733,2013-01-03,2013-02-04,2013-02-04,2013-12-20,2023-05-01,NaT,2017-02-03,2013-07-31,2020-07-31,2017-02-24,NaT,NaT,NaN,3E Renewable Energy Consulting,NaN,Plan Insurance Brokers,Peroptyx,None,Software Developer,Data Mining Intern,Data Mining Intern,Software Developer,Data Analyst - Map Applications,None
3548,15816,2016-06-10,2019-06-01,2020-01-01,2022-02-01,NaT,NaT,2016-08-11,2019-12-09,2021-10-19,NaT,NaT,NaT,Neptune Oil S.A,Stoneshed FX,AMA Consultants,CAMTEL (Cameroon Telecommunications),None,None,Administrative assinstant (Intern),Financial Analyst (Intern),Office manager,Administrative and Financial Inspector,None,None
3611,15922,2015-09-06,2017-06-12,2018-06-10,2018-06-10,NaT,NaT,2016-05-20,2017-07-07,2018-08-09,2020-08-20,NaT,NaT,Colibri Energy,Ashurst LLP,Jebsen & Jessen Hamburg,Jebsen & Jessen Hamburg,None,None,Sales Associate,Partner Assistant,Controlling & Accounting,Controlling & Accounting,None,None
4115,17002,2018-09-01,2019-02-25,2019-09-01,2020-02-01,NaT,NaT,2018-11-30,2019-06-30,2020-01-15,NaT,NaT,NaT,Mercedes-Benz Korea,Prettl Produktions Holding GmbH,Mercedes-Benz AG,"Tec de Monterrey, Campus Guadalajara, Mexico",None,None,Intern Sales & Marketing,Working Student,Bachelor Thesis Student,Exchange Student,None,None


The cells above lead us to a different conclusion and a new set of actions that we did not previously consider. A person can have multiple jobs at a time, and they can quit one that they started later on in their lives and still maintain the one they already had. Therefore, it may be incorrect to fill the missing values of `DataFim` with the `DataInicio` of the following experience, and the missing values of `DataInicio` with the `DataFim` of the the past experience, as we intended to do, since it may introduce incorrect information in our data, and transmit an idea that is not the one actual reality depicits.

Taking this into consideration, __we will, for the time being, move on with these missing values__.

Finally, let us dig into the missing values of our df_students dataset.

-------
### <font color='#BFD62F'>3.2.3. Students' Data</font> <a class="anchor" id="studentsmissing"></a>
[Back to Contents](#toc)


In [45]:
df_students.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3416 entries, 0 to 3415
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id_individuo   3416 non-null   int64         
 1   nm_curso_pt    3416 non-null   object        
 2   DT_MATRIC      3416 non-null   datetime64[ns]
 3   dataConclusao  3416 non-null   datetime64[ns]
 4   NotaFinal      3416 non-null   float64       
 5   situacaoFinal  3416 non-null   object        
 6   cd_curso       3416 non-null   int64         
dtypes: datetime64[ns](2), float64(1), int64(2), object(2)
memory usage: 186.9+ KB


Amazing. There are no missing values in the df_students dataset. We will check the variables to ensure that there are no masked missing values, but, if not, we can move on to the the next step of our preprocessing stage.

In [46]:
tf.print_all_values(df_students, "nm_curso_pt")

Mestrado em Gestão de Informação || Mestrado em Ciência de Dados e Métodos Analíticos Avançados || Pós-Graduação em Enterprise Data Science and Analytics || Mestrado em Marketing Analítico (Data Driven Marketing) || Mestrado em Estatística e Gestão de Informação
Pós-Graduação em Digital Marketing and Analytics || Pós-Graduação em Gestão de Informação e Business Intelligence na Saúde || Pós-Graduação em Data Science for Finance || Pós-Graduação em Gestão e Controlo Financeiro e Orçamental || Pós-Graduação em Business Intelligence
Pós-Graduação em Digital Enterprise Management || Pós-Graduação em Gestão de Informações e Segurança || Pós-Graduação em Gestão dos Sistemas de Informação || Pós-Graduação em Mercados e Riscos Financeiros || Pós-Graduação em Data Science for Marketing
Pós-Graduação em Marketing Intelligence || Pós-Graduação em Sistemas de Informação Empresariais || Pós-Graduação em Sistemas Estatísticos || Pós-Graduação em Análise e Gestão de Risco || Post-graduate program in B

In [47]:
tf.print_all_values(df_students, "DT_MATRIC")

2021-03-17 00:00:00 || 2022-03-29 00:00:00 || 2020-03-27 00:00:00 || 2019-03-26 00:00:00 || 2018-05-22 00:00:00
2023-01-30 00:00:00 || 2022-03-30 00:00:00 || 2021-01-05 00:00:00 || 2018-09-04 00:00:00 || 2022-02-02 00:00:00
2019-09-05 00:00:00 || 2020-03-26 00:00:00 || 2023-07-24 00:00:00 || 2019-03-25 00:00:00 || 2019-05-21 00:00:00
2021-05-25 00:00:00 || 2019-01-02 00:00:00 || 2021-05-24 00:00:00 || 2022-06-07 00:00:00 || 2022-07-28 00:00:00
2020-01-30 00:00:00 || 2018-03-23 00:00:00 || 2019-02-06 00:00:00 || 2022-06-06 00:00:00 || 2021-07-27 00:00:00
2020-07-28 00:00:00 || 2018-09-05 00:00:00 || 2020-12-03 00:00:00 || 2021-01-29 00:00:00 || 2020-01-28 00:00:00
2019-07-15 00:00:00 || 2020-06-01 00:00:00 || 2020-07-27 00:00:00 || 2020-01-31 00:00:00 || 2019-02-01 00:00:00
2019-04-09 00:00:00 || 2023-02-01 00:00:00 || 2022-07-18 00:00:00 || 2020-05-29 00:00:00 || 2020-04-08 00:00:00
2020-05-28 00:00:00 || 2021-03-19 00:00:00 || 2018-11-21 17:45:57 || 2019-09-06 00:00:00 || 2022-02-04 0

In [48]:
tf.print_all_values(df_students, "dataConclusao")

2023-02-14 00:00:00 || 2024-10-31 00:00:00 || 2024-10-28 00:00:00 || 2024-10-29 00:00:00 || 2024-10-30 00:00:00
2022-01-27 00:00:00 || 2023-01-26 00:00:00 || 2020-06-27 00:00:00 || 2023-01-24 00:00:00 || 2024-01-23 00:00:00
2023-01-27 00:00:00 || 2023-01-23 00:00:00 || 2021-01-22 00:00:00 || 2023-01-25 00:00:00 || 2024-01-29 00:00:00
2024-01-30 00:00:00 || 2023-10-25 00:00:00 || 2021-01-20 00:00:00 || 2022-02-11 00:00:00 || 2023-10-23 00:00:00
2024-02-01 00:00:00 || 2022-04-21 00:00:00 || 2023-10-24 00:00:00 || 2024-02-02 00:00:00 || 2023-04-10 00:00:00
2020-07-21 00:00:00 || 2024-07-01 00:00:00 || 2020-08-30 00:00:00 || 2024-01-31 00:00:00 || 2022-01-26 00:00:00
2021-06-25 00:00:00 || 2022-07-18 00:00:00 || 2020-01-22 00:00:00 || 2022-07-01 00:00:00 || 2022-07-13 00:00:00
2023-10-27 00:00:00 || 2019-07-19 00:00:00 || 2021-06-24 00:00:00 || 2024-02-05 00:00:00 || 2019-09-20 00:00:00
2024-02-06 00:00:00 || 2019-07-05 00:00:00 || 2019-06-28 00:00:00 || 2022-01-28 00:00:00 || 2023-07-10 0

In [49]:
tf.print_all_values(df_students, "NotaFinal")

17.0 || 16.0 || 15.0 || 18.0 || 14.0
13.0 || 19.0 || 12.0 || 20.0 || 14.1
15.25 || 16.14 || 11.0 || 18.63 || 14.75
13.8 || 15.19 || 17.08 || 14.07 || 16.23
17.62 || 17.35 || 14.86 || 13.92 || 14.26
15.43 || 14.35 || 15.58 || 16.88 || 15.42
17.88 || 14.13 || 15.13 || 15.2 || 16.38
14.65 || 16.08 || 13.38 || 16.71 || 17.2
13.25 || 13.5 || 14.59 || 16.1 || 16.76
16.68 || 13.88 || 14.5 || 15.31 || 16.65
15.28 || 17.63 || 16.75 || 15.11 || 15.88
16.2

Unique values for 'NotaFinal': 56

Missing values for 'NotaFinal': 0


## <font color='#BFD62F'>3.3. Removing Outliers</font> <a class="anchor" id="outliers"></a>
[Back to Contents](#toc)

For the time being, the data that we have is not prone to the display of outliers (the exception could be the `NotaFinal` column of our df_students table). Therefore, we will skip this step (for now), and move on to feature engineering, where we will transform the data into (hopefully) more relevant information, and where outliers may be more easily identified.

## <font color='#BFD62F'>3.4. Feature Engineering</font> <a class="anchor" id="engineering"></a>
[Back to Contents](#toc)

-------
### <font color='#BFD62F'>3.4.1. Creating New Features </font> <a class="anchor" id="newfeatures"></a>
[Back to Contents](#toc)

This is the space to create some new features which may be relevant - some were already introduced in the exploration stage, while others may now be created as a consequence of our understanding of the data. For now, we will focus on creating as many variables as we think that might be relevant for the models to use. Later, we will perform feature selection (for each specific task) to identify the most relevant features for each situation.

In [50]:
df_applicants.head()

,idCandidatura,DGrauAcademicoPT_1,DGrauAcademicoPT_2,DGrauAcademicoPT_3,DEstabEnsinoProvPT_1,DEstabEnsinoProvPT_2,DEstabEnsinoProvPT_3,DCursoProvPT_1,DCursoProvPT_2,DCursoProvPT_3,CursoConcluido_1,CursoConcluido_2,CursoConcluido_3,NotaOuMedia_1,NotaOuMedia_2,NotaOuMedia_3,DescEscala_1,DescEscala_2,DescEscala_3,ID_INDIVIDUO,idcandidato,idgenero,cd_pais,Localidade,nivelInglesCompreensao,nivelInglesFala,nivelInglesEscrita,datacandidaturafim,dprograma,dcurso,DCursoOpcao,idEstadoCandidatura,DEstadoPT
0,6724,Licenciado,None,None,Coventry University,None,None,BA (Hons) Business Management,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,8855,M,CM,COVENTRY,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),2018-01-07 12:30:47.150,Post-Graduate,Postgraduate program,Enterprise Information Systems,4,Completa
1,6726,Licenciado,None,None,Anhanguera educacional,None,None,Engenharia de Produção Mecânica,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,8858,M,BR,taubate,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),2018-01-07 21:29:53.603,Pós-Graduação,Pós-Graduação,Sistemas de Informação Empresariais,4,Completa
2,6729,Sem Grau,None,None,Escola Secundaria Siumara Da Costa Primo,None,None,Ciências Sociais,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,8832,F,AO,Luanda,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),2018-01-08 11:30:37.810,Pós-Graduação,Pós-Graduação,Sistemas Estatísticos - Especialização em Estatísticas de Bancos Centrais,2,Em Análise
3,6731,Licenciado,None,None,Delhi university,None,None,BA Pass,None,None,S,None,None,45,None,None,Minimal positive: 35% | Maximum: 100%,None,None,NaN,8833,M,PT,amadora,Fluente (QERC: C2),Fluente (QERC: C2),Fluente (QERC: C2),2018-01-08 16:23:42.327,Pós-Graduação,Pós-Graduação,Sistemas de Informação Empresariais,4,Completa
4,6737,Licenciado,None,None,UAN,None,None,Engenharia da Gestão e Ordenamento,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,7016,M,AO,Luanda,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),2018-01-09 16:00:56.433,Pós-Graduação,Pós-Graduação,Inteligência Geoespacial,4,Completa


#### Number of Academic Experiences and Degrees

The first variable that we will add to our dataset will return the total number of academic achievements mentioned by the candidate upon its application. Similarly, we will also create another attribute that will return the number of academic degrees attained by the applicant (excluding "Ensino Médio" and "Sem Grau", but including "Finalista").

We expect these two variables to be highly correlated. However, we will create both at the moment and leave to the feature selection stage the decision on which one to keep.

In [51]:
df_applicants["NumberAcademicExperiences"] = df_applicants[
    ["DGrauAcademicoPT_1", "DGrauAcademicoPT_2", "DGrauAcademicoPT_3"]].notna().sum(axis = 1)

df_applicants["NumberAcademicDegrees"] = df_applicants[["DGrauAcademicoPT_1", "DGrauAcademicoPT_2", "DGrauAcademicoPT_3"]].apply(
    lambda x: x[~x.isin(["Ensino Médio", "Sem Grau"])].notna().sum(), axis = 1)

#### Highest Academic Degree

From the `DGrauAcademicoPT` variables, we can also retrieve the highest academic degree that a certain applicant has attained. This will be the object of our new attribute, `HighestAcademicDegree`.

In [52]:
degree_levels = ["Doutorado", "Mestre", "Licenciado", "Finalista", "Ensino Médio", "Sem Grau"]

df_applicants["HighestAcademicDegree"] = df_applicants[["DGrauAcademicoPT_1", "DGrauAcademicoPT_2", "DGrauAcademicoPT_3"]].apply(
    tf.max_academic_degree, axis = 1, ordered_list = degree_levels)

#### University Rankings

Using the `DGrauAcademicoPT` variables, we can attempt to assess the quality of the previous academic institutions attended by the applicant by retrieving the HEI rankings. In our case, we will be using the most recent [Times Higher Education](https://www.timeshighereducation.com/) HEI ranking at the time of our study, referring to the year of 2025, which also returns a group of potentially relevant indicators, such as the representation of international students and the female-to-male ratio, along with other quality metrics. It contains more than 2 000 HEIs worldwide.

The dataset to be imported was retrieved from [Kaggle](https://www.kaggle.com/datasets/raymondtoo/the-world-university-rankings-2016-2024) and contains the rankings between 2016 and 2025 - we will only keep data for the most recent period available.

In [53]:
rankings = pd.read_csv("THE World University Rankings 2016-2025.csv")
rankings = rankings[rankings["Year"] == 2025].drop(columns = {"Year"})

rankings.head()

,Rank,Name,Country,Student Population,Students to Staff Ratio,International Students,Female to Male Ratio,Overall Score,Teaching,Research Environment,Research Quality,Industry Impact,International Outlook
12430,1.0,University of Oxford,United Kingdom,22095.0,10.8,43%,51 : 49,98.4775,96.8,100.0,98.8,99.6,97.3
12431,2.0,Massachusetts Institute of Technology,United States,11836.0,8.0,33%,42 : 58,98.0490,99.2,96.0,99.7,100.0,93.8
12432,3.0,Harvard University,United States,22584.0,10.0,25%,52 : 48,97.6500,97.3,99.9,99.3,85.7,90.1
12433,4.0,Princeton University,United States,8378.0,7.8,23%,47 : 53,97.5195,98.3,98.0,98.9,96.9,87.4
12434,5.0,University of Cambridge,United Kingdom,20980.0,11.5,38%,49 : 51,97.3600,95.9,99.9,97.6,88.4,97.1


In [54]:
rankings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2092 entries, 12430 to 14521
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Rank                     2092 non-null   float64
 1   Name                     2092 non-null   object 
 2   Country                  2092 non-null   object 
 3   Student Population       2092 non-null   float64
 4   Students to Staff Ratio  2092 non-null   float64
 5   International Students   2092 non-null   object 
 6   Female to Male Ratio     2011 non-null   object 
 7   Overall Score            2092 non-null   float64
 8   Teaching                 2092 non-null   float64
 9   Research Environment     2092 non-null   float64
 10  Research Quality         2092 non-null   float64
 11  Industry Impact          2092 non-null   float64
 12  International Outlook    2092 non-null   float64
dtypes: float64(9), object(4)
memory usage: 228.8+ KB


Before adding this information to our data, we could transform 2 variables that are currently passed as strings, but are very well convertible to numbers: `International Students` and `Female to Male Ratio`.

In [55]:
tf.print_all_values(rankings, "International Students")

0% || 1% || 2% || 3% || 5%
4% || 6% || 8% || 7% || 13%
12% || 9% || 10% || 11% || 14%
15% || 18% || 16% || 17% || 20%
24% || 22% || 19% || 21% || 27%
26% || 31% || 29% || 30% || 32%
38% || 23% || 25% || 28% || 33%
43% || 37% || 34% || 46% || 45%
35% || 41% || 36% || 39% || 40%
44% || 49% || 51% || 50% || 48%
52% || 61% || 55% || 42% || 54%
70% || 60% || 47% || 74% || 86%
62% || 75% || 78% || 88% || 72%
93% || 81% || 69% || 67% || 64%
53% || 71% || 58% || 66%

Unique values for 'International Students': 74

Missing values for 'International Students': 0


In [56]:
tf.print_all_values(rankings, "Female to Male Ratio")

55 : 45 || 56 : 44 || 54 : 46 || 57 : 43 || 58 : 42
52 : 48 || 60 : 40 || 59 : 41 || 61 : 39 || 50 : 50
51 : 49 || 53 : 47 || 47 : 53 || 49 : 51 || 63 : 37
62 : 38 || 64 : 36 || 43 : 57 || 48 : 52 || 38 : 62
45 : 55 || 46 : 54 || 37 : 63 || 65 : 35 || 66 : 34
39 : 61 || 41 : 59 || 44 : 56 || 42 : 58 || 67 : 33
35 : 65 || 40 : 60 || 32 : 68 || 30 : 70 || 34 : 66
36 : 64 || 70 : 30 || 68 : 32 || 27 : 73 || 31 : 69
28 : 72 || 33 : 67 || 24 : 76 || 25 : 75 || 69 : 31
29 : 71 || 71 : 29 || 26 : 74 || 100 : 0 || 74 : 26
72 : 28 || 73 : 27 || 18 : 82 || 75 : 25 || 22 : 78
20 : 80 || 23 : 77 || 21 : 79 || 13 : 87 || 19 : 81
77 : 23 || 76 : 24 || 17 : 83 || 12 : 88 || 14 : 86
11 : 89 || 85 : 15 || 79 : 21 || 9 : 91 || 2 : 98
8 : 92 || 80 : 20 || 86 : 14 || 10 : 90 || 16 : 84
0 : 100 || 97 : 3 || 15 : 85

Unique values for 'Female to Male Ratio': 79

Missing values for 'Female to Male Ratio': 81


In [57]:
rankings["Percentage_International_Students"] = rankings["International Students"].str.rstrip("%").astype(int)
rankings["Percentage_Female_Students"] = rankings["Female to Male Ratio"].str.extract(r"(\d+)").astype(float, errors = "ignore")

rankings.drop(columns = {"International Students", "Female to Male Ratio"}, inplace = True)

For our merge, we have to consider that applicants do not always write the institutional name of the HEI in the English Language, in accordance with the Times Higher Education ranking. Therefore, we will perform a fuzzy merge, assuming that, if more than 80% of the content matches, then there is a correspondence between the two values and they refer to the same HEI. Although this may introduce some noise in our data from incorrect matches, we believe that this is a better option than performing an exact match, which would fail to identify correct relationships between both datasets in the vast majority of the observations.

Note: Although it is also mentioned in the functions file, this function was created with the help of ChatGPT.

In [58]:
df_applicants = tf.fuzzy_merge(df_applicants, rankings, "DEstabEnsinoProvPT_1", "Name", "_1", 80)
df_applicants = tf.fuzzy_merge(df_applicants, rankings, "DEstabEnsinoProvPT_2", "Name", "_2", 80)
df_applicants = tf.fuzzy_merge(df_applicants, rankings, "DEstabEnsinoProvPT_3", "Name", "_3", 80)

df_applicants.drop(columns = ["Name_1", "Name_2", "Name_3"], inplace = True)
df_applicants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14812 entries, 0 to 14811
Data columns (total 72 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   idCandidatura                        14812 non-null  int64         
 1   DGrauAcademicoPT_1                   14812 non-null  object        
 2   DGrauAcademicoPT_2                   4282 non-null   object        
 3   DGrauAcademicoPT_3                   923 non-null    object        
 4   DEstabEnsinoProvPT_1                 14752 non-null  object        
 5   DEstabEnsinoProvPT_2                 4268 non-null   object        
 6   DEstabEnsinoProvPT_3                 916 non-null    object        
 7   DCursoProvPT_1                       14747 non-null  object        
 8   DCursoProvPT_2                       4267 non-null   object        
 9   DCursoProvPT_3                       911 non-null    object        
 10  CursoConcl

Since we are not so interested in the characteristics of each individual HEI the applicant attended, but rather in overall environment they studied at, we will take the average of each indicator (including ranking) and store it as a variable, eliminating those that concern specific institutions.

In [59]:
hei_columns = ["Rank_1", "Student Population_1", "Students to Staff Ratio_1", "Overall Score_1", "Teaching_1", "Research Environment_1",
               "Research Quality_1", "Industry Impact_1", "International Outlook_1", "Percentage_International_Students_1",
               "Percentage_Female_Students_1", "Rank_2", "Student Population_2", "Students to Staff Ratio_2", "Overall Score_2", "Teaching_2",
               "Research Environment_2", "Research Quality_2", "Industry Impact_2", "International Outlook_2",
               "Percentage_International_Students_2", "Percentage_Female_Students_2", "Rank_3", "Student Population_3",
               "Students to Staff Ratio_3", "Overall Score_3", "Teaching_3", "Research Environment_3", "Research Quality_3",
               "Industry Impact_3", "International Outlook_3", "Percentage_International_Students_3", "Percentage_Female_Students_3"]

prefix_to_columns = {}
for col in hei_columns:
    prefix = "_".join(col.split("_")[:-1])
    if prefix not in prefix_to_columns:
        prefix_to_columns[prefix] = []
    prefix_to_columns[prefix].append(col)

In [60]:
for prefix, cols in prefix_to_columns.items():
    df_applicants[prefix] = df_applicants[cols].mean(axis = 1)

df_applicants.drop(columns = hei_columns, inplace = True)
df_applicants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14812 entries, 0 to 14811
Data columns (total 50 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   idCandidatura                      14812 non-null  int64         
 1   DGrauAcademicoPT_1                 14812 non-null  object        
 2   DGrauAcademicoPT_2                 4282 non-null   object        
 3   DGrauAcademicoPT_3                 923 non-null    object        
 4   DEstabEnsinoProvPT_1               14752 non-null  object        
 5   DEstabEnsinoProvPT_2               4268 non-null   object        
 6   DEstabEnsinoProvPT_3               916 non-null    object        
 7   DCursoProvPT_1                     14747 non-null  object        
 8   DCursoProvPT_2                     4267 non-null   object        
 9   DCursoProvPT_3                     911 non-null    object        
 10  CursoConcluido_1                  

We will fill the missing values for the `Rank` variable with 2 100, to identify HEIs that fall off the Times Higher Education classification of the best 2 093 HEIs worldwide, but we will first create an auxiliary variable to tell us if any of the institutions previously attended by the applicant originally appeared in the ranking.

In [61]:
df_applicants["Is_Ranked"] = df_applicants["Rank"].notna().astype(int)
df_applicants[["Rank"]] = df_applicants[["Rank"]].fillna(2100)

As for the remaining missing values, although their presence is more than justified, we can not apply models to the dataset unless we fill them in some form. To that extent, we will make a __very strong assumption that students from the same country study in the same countries as well__, and we will fill these values with their respective country's medians. Where this is not possible, we will impute the global median values. In the worst case scenario, these variables will be eliminated in the feature selection stage, and we lose the chance to make use of this information.

Notes:
1) An alternative could be to use algorithms such as KNN Imputer to fill the missing values. However, this would imply assuming that students with similar characteristics study in universities with similar characteristics as well, which could be seen as an even bolder assumption. Moreover, the sample of neighbours would be very low for most of variables, increasing the risk of imputing wrong information.
2) We will not bother to fill the missing values for `Country_1`, `Country_2` and `Country_3`, since these are text variables that will be dropped at the end of the feature engineering stage.

In [62]:
# Updating the list of HEI columns
hei_columns = ["Student Population", "Students to Staff Ratio", "Overall Score", "Teaching", "Research Environment",
               "Research Quality", "Industry Impact", "International Outlook", "Percentage_International_Students",
               "Percentage_Female_Students"]

for col in hei_columns:
    df_applicants[col] = df_applicants.groupby("cd_pais")[col].transform(lambda x: x.fillna(x.median()))

df_applicants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14812 entries, 0 to 14811
Data columns (total 51 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   idCandidatura                      14812 non-null  int64         
 1   DGrauAcademicoPT_1                 14812 non-null  object        
 2   DGrauAcademicoPT_2                 4282 non-null   object        
 3   DGrauAcademicoPT_3                 923 non-null    object        
 4   DEstabEnsinoProvPT_1               14752 non-null  object        
 5   DEstabEnsinoProvPT_2               4268 non-null   object        
 6   DEstabEnsinoProvPT_3               916 non-null    object        
 7   DCursoProvPT_1                     14747 non-null  object        
 8   DCursoProvPT_2                     4267 non-null   object        
 9   DCursoProvPT_3                     911 non-null    object        
 10  CursoConcluido_1                  

In [63]:
for col in hei_columns:
    df_applicants[col] = df_applicants[col].fillna(df_applicants[col].median())

#### Previous IMS Student

As we saw in the exploration phase of our data, having been a prior Nova IMS student might be a valuable predictor of admission and academic performance. Therefore, we will create a new binary variable `Previous_IMS_Student` to reflect this.

In [64]:
df_applicants = tf.create_conditional_column(
    df = df_applicants,
    columns_to_find = ["DEstabEnsinoProvPT_1", "DEstabEnsinoProvPT_2", "DEstabEnsinoProvPT_3"],
    new_column = "Previous_IMS_Student",
    values_to_find = ["Universidade Nova de Lisboa - Instituto Superior de Estatística e Gestão de Informação", "IMS",
                      "Information Management School", "ISEGI"],
    positive_value = 1,
    negative_value = 0)

We should check if there are corrections to be made regarding the candidates that include some of the terms above that were more ambiguous.

In [65]:
df_applicants[df_applicants["DEstabEnsinoProvPT_1"].str.contains(r"IMS|Information Management School|ISEGI", case = False, na = False)][
    ["idCandidatura", "DEstabEnsinoProvPT_1", "DEstabEnsinoProvPT_2", "DEstabEnsinoProvPT_3", "Previous_IMS_Student"]
]

,idCandidatura,DEstabEnsinoProvPT_1,DEstabEnsinoProvPT_2,DEstabEnsinoProvPT_3,Previous_IMS_Student
2718,13516,Higher Institute of Management Studies (HIMS) Molyko-Buea Cameroon,Liaoning Shihua University Fushun City China,None,1
3433,15449,Noca IMS,None,None,1
3878,15455,Noca IMS,None,None,1
4270,16864,HIGHER INSTITUTE OF MANAGEMENT STUDIES (HIMS) BUEA - MOLYKO,None,None,1
4871,18386,IMS UNISON,None,None,1
6164,21249,NOVA IMS - Information Management School,None,None,1
8272,26262,Nova IMS,Escola Superior de Hotelaria e Turismo do Estoril,None,1
8337,26458,NOVA IMS/ IFGI Munster/ Jaume I University,None,None,1
8357,26474,Universidade NOVA / NOVA IMS,None,None,1
8359,26476,Universidade NOVA / NOVA IMS,None,None,1


In [66]:
df_applicants[df_applicants["DEstabEnsinoProvPT_2"].str.contains(r"IMS|Information Management School|ISEGI", case = False, na = False)][
    ["idCandidatura", "DEstabEnsinoProvPT_1", "DEstabEnsinoProvPT_2", "DEstabEnsinoProvPT_3", "Previous_IMS_Student"]
]

,idCandidatura,DEstabEnsinoProvPT_1,DEstabEnsinoProvPT_2,DEstabEnsinoProvPT_3,Previous_IMS_Student
550,8322,Universidade Nova de Lisboa - Faculdade de Ciências e Tecnologia,NovaIMS,None,1
1152,9657,Universidade Técnica de Lisboa - Instituto Superior de Agronomia,NOVA Information management School (IMS),None,1
5295,19332,Universidade de Coimbra - Faculdade de Letras,IMS,None,1
5789,20634,Vilnius University,NOVA IMS Information Management School,San Diego State University,1
5990,21130,Universidade Nova de Lisboa - Faculdade de Ciências e Tecnologia,Universidade Nova de Lisboa - IMS,None,1
6289,21894,"European School of Business (ESB) in Reutlingen, Germany","NEOMA Business School in Reims, France",None,1
6567,21880,"European School of Business (ESB) in Reutlingen, Germany","NEOMA Business School in Reims, France",None,1
7325,9750,Universidade Técnica de Lisboa - Instituto Superior de Agronomia,NOVA Information management School (IMS),None,1
7597,24682,Unifersidade Federal de Minas Gerais,NOVA IMS - Information Management School | NOVA IMS,None,1
9073,27609,Universidade Técnica de Lisboa - Instituto Superior Técnico,NOVA IMS,None,1


In [67]:
df_applicants[df_applicants["DEstabEnsinoProvPT_3"].str.contains(r"IMS|Information Management School|ISEGI", case = False, na = False)][
    ["idCandidatura", "DEstabEnsinoProvPT_1", "DEstabEnsinoProvPT_2", "DEstabEnsinoProvPT_3", "Previous_IMS_Student"]
]

,idCandidatura,DEstabEnsinoProvPT_1,DEstabEnsinoProvPT_2,DEstabEnsinoProvPT_3,Previous_IMS_Student
83,7239,Universidade Federal de Goiás,Instituto Nacional de Telecomunicações,Nova IMS,1
1846,11679,Higher institute of management studies,Cambridge Institute of Professional Training,Higher Institute of Management Studies (HIMS) Buea Cameroon,1
4529,8118,Universidade Federal de Goiás,Instituto Nacional de Telecomunicações,Nova IMS,1
5388,19599,"British Council, Sylhet, Bangladesh","Mornington University College, Bangladesh","AIMS academy, Bangladesh. Approved by OTHM, United Kingdom",1
6324,19675,"British Council, Sylhet, Bangladesh","Mornington University College, Bangladesh","AIMS academy, Bangladesh. Approved by OTHM, United Kingdom",1
8098,26001,Grinnell College,IE Business School,NOVA Information Management School,1
8131,25971,Grinnell College,IE Business School,NOVA Information Management School,1
10260,31584,AL QURAN BEACON COLLEGE,AL QURAN BEACON SCHOOL,4 year from Arid agriculture university department of (UIMS),1


In [68]:
applications_to_correct = [13516, 15864, 18386, 21880, 21894, 29525, 11679, 19599, 19675, 31584]

df_applicants.loc[df_applicants["idCandidatura"].isin(applications_to_correct), "Previous_IMS_Student"] = 0

#### Previous Studies

We also explored the possibility that certain students with an academic background in certain specified fields could be more (or less) likely to be admitted and have a better (or worse academic performance). With that in mind, we will create new attributes using the `DCursoProvPT` columns as the basis of our transformation.

In [69]:
study_fields = {
    "Previous_Data_Studies": ["Dados", "Data", "Informação", "Information"],
    "Previous_Computer_Studies": ["Comput", "Software", "Informática", "Informatics"],
    "Previous_Tech_Studies": ["Tech", "Tecno", "IT"],
    "Previous_Math_Studies": ["Math", "Mat", "Estat", "Stat"],
    "Previous_Engineering_Studies": ["Engenharia", "Engineering"],
    "Previous_Finance_Studies": ["Finan"],
    "Previous_Marketing_Studies": ["Marketing", "Mkt"]}

for column_name, terms in study_fields.items():
    df_applicants = tf.create_conditional_column(
        df = df_applicants,
        columns_to_find = ["DCursoProvPT_1", "DCursoProvPT_2", "DCursoProvPT_3"],
        new_column = column_name,
        values_to_find = terms,
        positive_value = 1,
        negative_value = 0)

We will aggregate the information provided by these variables in two new columns: `Previous_Field_Studies` will be 1 if the applicant studied any of the mentioned fields above, while `Number_Studied_Fields` will sum the number of studied fields. We expect these variables to be strongly correlated, but we will keep both for the time being and leave the decision on which to keep to the feature selection stage.

In [70]:
df_applicants["Previous_Field_Studies"] = df_applicants[list(study_fields.keys())].any(axis=1).astype(int)
df_applicants["Number_Studied_Fields"] = df_applicants[list(study_fields.keys())].sum(axis=1)

#### Number of Non-Finished Programs and Proportion of Concluded Programs

We can transform the `CursoConcluido` variables into a count of the programs that were taken but not completed by the candidate at the time of the application. Following a similar logic, we can also retrieve the proportion of concluded programs, also using the `NumberAcademicAchievements` attribute created earlier.

In [71]:
df_applicants["Non_Finished_Programs"] = df_applicants[["CursoConcluido_1", "CursoConcluido_2", "CursoConcluido_3"]].apply(
    lambda row: str(sum(row == "N")), axis = 1).astype(int)

df_applicants["Prop_Finished_Programs"] = (
    np.round(
        np.where(
            df_applicants["NumberAcademicExperiences"] == 0, 0,
            1 - (df_applicants["Non_Finished_Programs"].astype(int) / df_applicants["NumberAcademicExperiences"].astype(int))
        ), 2))

#### Prior Grades

As we were able to understand in the exploration phase, one of our main obstacles was to accurately retrieve the grades achieved by the applicants in their previous academic endeavours. Now, we will attempt to extract the grade from the text, along with the scale of that same grade, so as to allow us to compute a scaled graded that can more easily be compare among the applicants.

In [72]:
df_applicants = tf.convert_to_numeric(df_applicants, "NotaOuMedia_1")
df_applicants = tf.convert_to_numeric(df_applicants, "NotaOuMedia_2")
df_applicants = tf.convert_to_numeric(df_applicants, "NotaOuMedia_3")

df_applicants = tf.extract_scale(df_applicants, "DescEscala_1", "MinimoEscala_1", "MaximoEscala_1")
df_applicants = tf.extract_scale(df_applicants, "DescEscala_2", "MinimoEscala_2", "MaximoEscala_2")
df_applicants = tf.extract_scale(df_applicants, "DescEscala_3", "MinimoEscala_3", "MaximoEscala_3")

df_applicants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14812 entries, 0 to 14811
Data columns (total 69 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   idCandidatura                      14812 non-null  int64         
 1   DGrauAcademicoPT_1                 14812 non-null  object        
 2   DGrauAcademicoPT_2                 4282 non-null   object        
 3   DGrauAcademicoPT_3                 923 non-null    object        
 4   DEstabEnsinoProvPT_1               14752 non-null  object        
 5   DEstabEnsinoProvPT_2               4268 non-null   object        
 6   DEstabEnsinoProvPT_3               916 non-null    object        
 7   DCursoProvPT_1                     14747 non-null  object        
 8   DCursoProvPT_2                     4267 non-null   object        
 9   DCursoProvPT_3                     911 non-null    object        
 10  CursoConcluido_1                  

Now that we have successfully converted the grades to numerical values and extracted the numerical scale inherent to these grades (for the observations where this transformation was possible), we can calculate the scaled grades, facilitating the comparison among students.

We will also remove the rows where it is not possible to extract any scaled grade, since we will later use these variables and we find them of relevance to our classification and regression tasks.

In [73]:
df_applicants["ScaledGrade_1"] = np.where(
    df_applicants["NotaOuMedia_1"].isna() | df_applicants["MaximoEscala_1"].isna(),
    np.nan,
    round(np.divide(df_applicants["NotaOuMedia_1"],
                    df_applicants["MaximoEscala_1"],
                    out = np.zeros_like(df_applicants["NotaOuMedia_1"]),
                    where = df_applicants["MaximoEscala_1"] !=0), 2))

df_applicants["ScaledGrade_2"] = np.where(
    df_applicants["NotaOuMedia_2"].isna() | df_applicants["MaximoEscala_2"].isna(),
    np.nan,
    round(np.divide(df_applicants["NotaOuMedia_2"],
                    df_applicants["MaximoEscala_2"],
                    out = np.zeros_like(df_applicants["NotaOuMedia_2"]),
                    where = df_applicants["MaximoEscala_2"] !=0), 2))

df_applicants["ScaledGrade_3"] = np.where(
    df_applicants["NotaOuMedia_3"].isna() | df_applicants["MaximoEscala_3"].isna(),
    np.nan,
    round(np.divide(df_applicants["NotaOuMedia_3"],
                    df_applicants["MaximoEscala_3"],
                    out = np.zeros_like(df_applicants["NotaOuMedia_3"]),
                    where = df_applicants["MaximoEscala_3"] !=0), 2))

df_applicants.dropna(subset = ["ScaledGrade_1", "ScaledGrade_2", "ScaledGrade_3"], how = "all", inplace = True)
print("Number of rows in the dataset after dropping rows with missing values in all 3 scaled grades columns:", df_applicants.shape[0])

Number of rows in the dataset after dropping rows with missing values in all 3 scaled grades columns: 13913


Nice. We managed to keep almost 94% of our rows after extracting the grades.

Let's now check for cases where the scaled grades are bigger than 1 (which should not happen).

In [74]:
len(df_applicants[(df_applicants["ScaledGrade_1"] > 1) | (df_applicants["ScaledGrade_2"] > 1) | (df_applicants["ScaledGrade_3"] > 1)])

248

After exploring some of these observations, we observe that some grades were incorrectly passed (some are even dates), or the scale may have been incorrectly identified. Since the number of rows is not extremely significant and it would be burdensome (and unscalable) to fill manually correct these values, we will drop these applicants from our dataset.

In [75]:
df_applicants = df_applicants[~((df_applicants["ScaledGrade_1"] > 1) | (df_applicants["ScaledGrade_2"] > 1) | (df_applicants["ScaledGrade_3"] > 1))]

We can now calculate the average, maximum, and minimum scaled grades attained by each applicant. These features may be relevant since we are unlikely to be able to use the grades for each specific academic program, due to the significant presence of missing values in these columns (especially `NotaOuMedia_2` and `NotaOuMedia_3`). Nonetheless, we will keep all rows for the time being and delete them in the feature selection stage. 

In [76]:
df_applicants["AverageScaledGrade"] = df_applicants[["ScaledGrade_1", "ScaledGrade_2", "ScaledGrade_3"]].mean(axis = 1)
df_applicants["MaxScaledGrade"] = df_applicants[["ScaledGrade_1", "ScaledGrade_2", "ScaledGrade_3"]].max(axis = 1)
df_applicants["MinScaledGrade"] = df_applicants[["ScaledGrade_1", "ScaledGrade_2", "ScaledGrade_3"]].min(axis = 1)

In [77]:
df_applicants.head()

,idCandidatura,DGrauAcademicoPT_1,DGrauAcademicoPT_2,DGrauAcademicoPT_3,DEstabEnsinoProvPT_1,DEstabEnsinoProvPT_2,DEstabEnsinoProvPT_3,DCursoProvPT_1,DCursoProvPT_2,DCursoProvPT_3,CursoConcluido_1,CursoConcluido_2,CursoConcluido_3,NotaOuMedia_1,NotaOuMedia_2,NotaOuMedia_3,DescEscala_1,DescEscala_2,DescEscala_3,ID_INDIVIDUO,idcandidato,idgenero,cd_pais,Localidade,nivelInglesCompreensao,nivelInglesFala,nivelInglesEscrita,datacandidaturafim,dprograma,dcurso,DCursoOpcao,idEstadoCandidatura,DEstadoPT,NumberAcademicExperiences,NumberAcademicDegrees,HighestAcademicDegree,Country_1,Country_2,Country_3,Rank,Student Population,Students to Staff Ratio,Overall Score,Teaching,Research Environment,Research Quality,Industry Impact,International Outlook,Percentage_International_Students,Percentage_Female_Students,Is_Ranked,Previous_IMS_Student,Previous_Data_Studies,Previous_Computer_Studies,Previous_Tech_Studies,Previous_Math_Studies,Previous_Engineering_Studies,Previous_Finance_Studies,Previous_Marketing_Studies,Previous_Field_Studies,Number_Studied_Fields,Non_Finished_Programs,Prop_Finished_Programs,MinimoEscala_1,MaximoEscala_1,MinimoEscala_2,MaximoEscala_2,MinimoEscala_3,MaximoEscala_3,ScaledGrade_1,ScaledGrade_2,ScaledGrade_3,AverageScaledGrade,MaxScaledGrade,MinScaledGrade
3,6731,Licenciado,None,None,Delhi university,None,None,BA Pass,None,None,S,None,None,45.0,NaN,NaN,Minimal positive: 35% | Maximum: 100%,None,None,NaN,8833,M,PT,amadora,Fluente (QERC: C2),Fluente (QERC: C2),Fluente (QERC: C2),2018-01-08 16:23:42.327,Pós-Graduação,Pós-Graduação,Sistemas de Informação Empresariais,4,Completa,1,1,Licenciado,United States,NaN,NaN,1758.0,7044.0,12.1,20.9800,24.1,7.7,28.4,17.0,32.5,6.0,71.0,1,0,0,0,0,0,0,0,0,0,0,0,1.0,35.0,100.0,NaN,NaN,NaN,NaN,0.45,NaN,NaN,0.45,0.45,0.45
6,6739,Licenciado,None,None,Universidade Técnica de Lisboa - Instituto Superior de Economia e Gestão,None,None,Economia,None,None,S,None,None,12.0,NaN,NaN,Minímo positivo: 10 | Máximo: 20,None,None,14404.0,6719,M,PT,Lisboa,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),2018-01-09 22:09:18.147,Pós-Graduação,Pós-Graduação,Sistemas de Informação Empresariais,6,Admitido,1,1,Licenciado,NaN,NaN,NaN,2100.0,14080.0,14.8,32.5875,24.3,16.1,53.3,36.0,54.8,16.0,55.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,10.0,20.0,NaN,NaN,NaN,NaN,0.60,NaN,NaN,0.60,0.60,0.60
12,6766,Mestre,None,None,Universidade Lusíada,None,None,Arquitectura,None,None,S,None,None,14.0,NaN,NaN,Minímo positivo: 10 | Máximo: 20,None,None,NaN,8408,F,PT,Vila Franca de Xira,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),2018-01-15 21:14:12.810,Pós-Graduação,Pós-Graduação,Sistemas de Informação Empresariais,4,Completa,1,1,Mestre,Portugal,NaN,NaN,1550.0,11773.0,9.7,24.4525,24.3,9.7,31.3,20.9,56.6,17.0,55.0,1,0,0,0,1,0,0,0,0,1,1,0,1.0,10.0,20.0,NaN,NaN,NaN,NaN,0.70,NaN,NaN,0.70,0.70,0.70
14,6773,Licenciado,Mestre,None,Universidade de Lisboa,Universidade de Lisboa,None,Planeamento,Sistemas de Informação Geográfica e Modelação Territorial Apl Ordenamento d,None,S,N,None,13.0,15.0,NaN,Minímo positivo: 10 | Máximo: 20,Minímo positivo: 10 | Máximo: 20,None,NaN,8945,M,PT,Alfeizerão,Nativo ou bilingue,Nativo ou bilingue,Nativo ou bilingue,2018-01-19 13:04:12.207,Pós-Graduação,Pós-Graduação,Inteligência Geoespacial,6,Admitido,2,2,Mestre,NaN,NaN,NaN,2100.0,14080.0,14.8,32.5875,24.3,16.1,53.3,36.0,54.8,16.0,55.0,0,0,1,0,1,0,0,0,0,1,2,1,0.5,10.0,20.0,10.0,20.0,NaN,NaN,0.65,0.75,NaN,0.70,0.75,0.65
15,6774,Licenciado,None,None,Faculdade de Telêmaco Borba (FATEB),None,None,Engenharia de Produção,None,None,S,None,None,8.2,NaN,NaN,Minímo positivo: 5 | Máximo: 10,None,None,14453.0,8811,M,PT,Olhao,Fluente (QERC: C2),Avançado (QERC: C1),Intermédio (QERC: B1-B2),2018-01-19 17:42:50.177,Pós-Graduação,Pós-Graduação,Sistemas de Informação Empresariais,4,Completa,1,1,Licenciado,NaN,NaN,NaN,2100.0,14080.0,14.8,32.5875,24.3,16.1,53.3,36.0,54.8,16.0,55.0,0,0,0,0,0,0,1,0,0,1,1,0,1.0,5.0,10.0,NaN,NaN,NaN,NaN,0.82,NaN,NaN,0.82,0.82,0.82


#### Country Variables

From `cd_pais`, we can add to our data more potentially relevant geographical information about the origin of our applicants.

Adding to those we retrieved in the exploration stage, we will now also bring two other variables, in an effort to provide some form of encoding to the candidates' countries and use it in our work:
* __The distance of the applicants' country of origin to Portugal__, using data from the French center for research and expertise on global economy ([Centre d'Etudes Prospectives et d'Informations Internationales - CEPII](https://www.cepii.fr/CEPII/en/welcome.asp)). The data is avaliable for download [here](https://www.cepii.fr/cepii/en/bdd_modele/bdd_modele_item.asp?id=6).
* **The countries' GDP _per capita_ in current prices**, using 2023 values provided by the [International Monetary Fund (IMF)](https://www.imf.org/en/Home). The data is avaliable for download [here](https://www.imf.org/external/datamapper/NGDPDPC@WEO/OEMDC/ADVEC/WEOWORLD).

In [78]:
url = "https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/refs/heads/master/all/all.csv"

countries = pd.read_csv(url)
countries.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [79]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   name                      249 non-null    object 
 1   alpha-2                   248 non-null    object 
 2   alpha-3                   249 non-null    object 
 3   country-code              249 non-null    int64  
 4   iso_3166-2                249 non-null    object 
 5   region                    247 non-null    object 
 6   sub-region                247 non-null    object 
 7   intermediate-region       105 non-null    object 
 8   region-code               247 non-null    float64
 9   sub-region-code           247 non-null    float64
 10  intermediate-region-code  105 non-null    float64
dtypes: float64(3), int64(1), object(7)
memory usage: 21.5+ KB


In [80]:
countries[countries["sub-region"].isna()]

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
8,Antarctica,AQ,ATA,10,ISO 3166-2:AQ,NaN,NaN,NaN,NaN,NaN,NaN
217,"Taiwan, Province of China",TW,TWN,158,ISO 3166-2:TW,NaN,NaN,NaN,NaN,NaN,NaN


Let's fill these missing values.

In [81]:
tf.print_all_values(countries, "region")
print("\n\n")
tf.print_all_values(countries, "sub-region")

Africa || Americas || Europe || Asia || Oceania

Unique values for 'region': 6

Missing values for 'region': 2



Sub-Saharan Africa || Latin America and the Caribbean || Western Asia || Southern Europe || Northern Europe
South-eastern Asia || Polynesia || Eastern Europe || Southern Asia || Western Europe
Micronesia || Eastern Asia || Northern Africa || Australia and New Zealand || Melanesia
Northern America || Central Asia

Unique values for 'sub-region': 18

Missing values for 'sub-region': 2


In [82]:
countries.loc[countries["name"] == "Antarctica", "region"] = "Antarctica"
countries.loc[countries["name"] == "Antarctica", "sub-region"] = "Antarctica"

countries.loc[countries["name"] == "Taiwan, Province of China", "region"] = "Asia"
countries.loc[countries["name"] == "Taiwan, Province of China", "sub-region"] = "Eastern Asia"

In [83]:
countries[countries["alpha-2"].isna()]

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
153,Namibia,NaN,NAM,516,ISO 3166-2:NA,Africa,Sub-Saharan Africa,Southern Africa,2.0,202.0,18.0


In [84]:
countries["alpha-2"].fillna("NA", inplace = True)

After cleaning this data, we can now proceed to merge it with our df_applicants table.

In [85]:
countries.drop(columns = {"country-code", "iso_3166-2", "intermediate-region", "region-code", "sub-region-code", "intermediate-region-code"},
               inplace = True)

df_applicants = df_applicants.merge(countries.add_prefix("countries_"), how = "left", left_on = "cd_pais", right_on = "countries_alpha-2"
).drop(columns = ["countries_alpha-2"])

We also saw that we had some outdated country codes in our applicants' data. We should correct these values before moving forward.

In [86]:
df_applicants[df_applicants["countries_name"].isna()]

,idCandidatura,DGrauAcademicoPT_1,DGrauAcademicoPT_2,DGrauAcademicoPT_3,DEstabEnsinoProvPT_1,DEstabEnsinoProvPT_2,DEstabEnsinoProvPT_3,DCursoProvPT_1,DCursoProvPT_2,DCursoProvPT_3,CursoConcluido_1,CursoConcluido_2,CursoConcluido_3,NotaOuMedia_1,NotaOuMedia_2,NotaOuMedia_3,DescEscala_1,DescEscala_2,DescEscala_3,ID_INDIVIDUO,idcandidato,idgenero,cd_pais,Localidade,nivelInglesCompreensao,nivelInglesFala,nivelInglesEscrita,datacandidaturafim,dprograma,dcurso,DCursoOpcao,idEstadoCandidatura,DEstadoPT,NumberAcademicExperiences,NumberAcademicDegrees,HighestAcademicDegree,Country_1,Country_2,Country_3,Rank,Student Population,Students to Staff Ratio,Overall Score,Teaching,Research Environment,Research Quality,Industry Impact,International Outlook,Percentage_International_Students,Percentage_Female_Students,Is_Ranked,Previous_IMS_Student,Previous_Data_Studies,Previous_Computer_Studies,Previous_Tech_Studies,Previous_Math_Studies,Previous_Engineering_Studies,Previous_Finance_Studies,Previous_Marketing_Studies,Previous_Field_Studies,Number_Studied_Fields,Non_Finished_Programs,Prop_Finished_Programs,MinimoEscala_1,MaximoEscala_1,MinimoEscala_2,MaximoEscala_2,MinimoEscala_3,MaximoEscala_3,ScaledGrade_1,ScaledGrade_2,ScaledGrade_3,AverageScaledGrade,MaxScaledGrade,MinScaledGrade,countries_name,countries_alpha-3,countries_region,countries_sub-region
1904,12390,Finalista,None,None,Faculty of Organizational Sciences,None,None,Operation Management,None,None,F,None,None,8.40,NaN,NaN,Minimal positive: 5 | Maximum: 10,None,None,16560.0,12871,F,CS,Belgrade,Intermédio (QERC: B1-B2),Intermédio (QERC: B1-B2),Intermédio (QERC: B1-B2),2019-03-13 19:45:36.987,Mestrado,Mestrado em Estatística e Gestão de Informação,Especialização em Marketing Research e CRM,6,Admitido,1,1,Finalista,NaN,NaN,NaN,2100.0,11836.0,18.5,22.28050,21.9,17.60,41.30,26.40,64.00,28.0,55.0,0,0,0,0,0,0,0,0,0,0,0,0,1.00,5.0,10.0,NaN,NaN,NaN,NaN,0.84,NaN,NaN,0.84,0.84,0.84,NaN,NaN,NaN,NaN
4403,18177,Licenciado,Licenciado,None,Lindenwood University,Lindenwood University,None,Análise Financeira,Finance,None,S,S,None,2.90,2.90,NaN,"Minimal positive: 0,5 | Maximum: 4","Minimal positive: 0,5 | Maximum: 4",None,19341.0,17019,F,CS,belgrade,Nativo ou bilingue,Nativo ou bilingue,Nativo ou bilingue,2020-05-18 19:18:03.523,Master,Master degree program in Information Management,Specialization in Information Systems and Technologies Management - Working Hours Schedule,10,Desistiu após seleção,2,2,Licenciado,Netherlands,Netherlands,NaN,74.0,34614.0,18.4,70.68350,46.8,62.80,93.80,98.70,87.70,22.0,60.0,1,0,0,0,0,0,0,1,0,1,1,0,1.00,0.5,4.0,0.5,4.0,NaN,NaN,0.72,0.72,NaN,0.72,0.72,0.72,NaN,NaN,NaN,NaN
8055,26921,Licenciado,None,None,"International Business School of Budapest , University Of Buckingham",None,None,"Bachelor’s of Science, Business with a specialization in Psychology",None,None,S,None,None,2.00,NaN,NaN,"' I Had upper division second class honors First-class honours – typically 70% or higher Second-class honours, upper division – typically 60 – 69% Second-class honours, lower division – typically 50 – 59% Third-class honours - typically 40 – 49%",None,None,23723.0,21993,F,YU,"Prishtine, Kosova",Fluente (QERC: C2),Fluente (QERC: C2),Fluente (QERC: C2),2022-03-21 16:19:24.223,Master,Master degree program in Data Science and Advanced Analytics,specialization in Business Analytics,6,Admitido,1,1,Licenciado,NaN,NaN,NaN,2100.0,14080.0,14.8,32.58750,24.3,16.10,53.30,36.00,54.80,16.0,55.0,0,0,0,0,1,0,0,0,0,1,1,0,1.00,40.0,70.0,NaN,NaN,NaN,NaN,0.03,NaN,NaN,0.03,0.03,0.03,NaN,NaN,NaN,NaN
8419,28090,Licenciado,None,None,The Altai State University,None,None,Specialist's degree in Advertising in Marketing,None,None,S,None,None,4.07,NaN,NaN,Minimal positive: 0 | Maximum: 5,None,None,27802.0,24366,M,CS,Belgrade,Intermédio (QERC: B1-B2),Intermédio (QERC: B1-B2),Intermédio (QERC: B1-B2),2022-05-26 16:25:55.520,Master,European Master of Science in Information Systems Management,European Master of Science in Inform

We will fill these missing values by assigning the cases where `cd_pais` is CS to Serbia, since all the locations referred (`Localidade`) are in this country. For the remaining observations, we will assign them to Kosovo (and replace `cd_pais` where it is "YU" by the Kosovar country code).

In [87]:
df_applicants.loc[df_applicants["cd_pais"] == "CS", "countries_name"] = "Serbia"
df_applicants.loc[df_applicants["cd_pais"] == "CS", "countries_alpha-3"] = "SRB"
df_applicants.loc[df_applicants["cd_pais"] == "CS", "countries_region"] = "Europe"
df_applicants.loc[df_applicants["cd_pais"] == "CS", "countries_sub-region"] = "Southern Europe"
df_applicants.loc[df_applicants["cd_pais"] == "CS", "cd_pais"] = "RS"

df_applicants.loc[df_applicants["cd_pais"] == "YU", "cd_pais"] = "XK"
df_applicants.loc[df_applicants["cd_pais"] == "XK", "countries_name"] = "Kosovo"
df_applicants.loc[df_applicants["cd_pais"] == "XK", "countries_alpha-3"] = "XKX"
df_applicants.loc[df_applicants["cd_pais"] == "XK", "countries_region"] = "Europe"
df_applicants.loc[df_applicants["cd_pais"] == "XK", "countries_sub-region"] = "Southern Europe"

Great! Now, we can proceed to import the distances.

In [88]:
distances = pd.read_excel("dist_cepii.xls", engine = "xlrd")
distances.head()

,iso_o,iso_d,contig,comlang_off,comlang_ethno,colony,comcol,curcol,col45,smctry,dist,distcap,distw,distwces
0,ABW,ABW,0,0,0,0,0,0,0,0,5.225315,5.225315,25.09354,23.04723
1,ABW,AFG,0,0,0,0,0,0,0,0,13257.810000,13257.810000,13168.22,13166.37
2,ABW,AGO,0,0,0,0,0,0,0,0,9516.913000,9516.913000,9587.316,9584.193
3,ABW,AIA,0,0,1,0,0,0,0,0,983.268200,983.268200,976.8974,976.8916
4,ABW,ALB,0,0,0,0,0,0,0,0,9091.742000,9091.742000,9091.576,9091.466


We are only interested in rows where the country of origin (given by `iso_o`) is Portugal (PRT) and in the bilateral distance (in kilometers) between the countries (given by `dist`).

In [89]:
distances = distances[distances["iso_o"] == "PRT"][["iso_d", "dist"]]
distances.head()

,iso_d,dist
36736,ABW,6637.424
36737,AFG,6791.342
36738,AGO,5779.503
36739,AIA,5655.651
36740,ALB,2472.398


Now that we have the distance between all countries and Portugal, we will add this to our df_applicants dataset.

In [90]:
df_applicants = df_applicants.merge(distances.add_prefix("countries_"), how = "left", left_on = "countries_alpha-3", right_on = "countries_iso_d"
).drop(columns = ["countries_iso_d"])

We are likely to have some values missing. Let's check which were the countries whose distance to Portugal is missing.

In [91]:
print("Number of missing distances:", df_applicants["countries_dist"].isna().sum())

df_applicants[df_applicants["countries_dist"].isna()]["countries_name"].unique()

Number of missing distances: 29


array(['Serbia', 'Romania', 'Kosovo', 'Montenegro', 'Timor-Leste'],
      dtype=object)

Source for filling the missing values: https://www.distancefromto.net/countries.php

In [92]:
df_applicants.loc[df_applicants["countries_name"] == "Serbia", "countries_dist"] = 2531.39
df_applicants.loc[df_applicants["countries_name"] == "Romania", "countries_dist"] = 2975.49
df_applicants.loc[df_applicants["countries_name"] == "Kosovo", "countries_dist"] = 2578.28
df_applicants.loc[df_applicants["countries_name"] == "Montenegro", "countries_dist"] = 2422.71
df_applicants.loc[df_applicants["countries_name"] == "Timor-Leste", "countries_dist"] = 14380.01

Let's now import the GDP per capita and add it to our df_applicants.

In [93]:
gdp = pd.read_excel("imf-dm-export-20241225.xls", engine = "xlrd")
gdp.head()

,"GDP per capita, current prices\n (U.S. dollars per capita)",1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029
0,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207.946,201.051,218.461,252.618,272.189,330.295,389.630,440.607,543.652,611.642,666.089,639.473,630.136,594.213,520.267,529.773,499.799,499.762,516.664,356.063,352.456,410.933,NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,728.359,817.734,824.542,815.529,788.332,788.801,855.724,832.245,805.046,860.784,675.736,408.022,259.750,452.623,736.028,903.989,1009.967,706.623,816.622,1037.599,1129.033,1283.237,1427.675,1829.632,2371.130,2673.773,2972.745,3595.053,4370.562,4114.090,4097.833,4439.890,4248.909,4415.600,4584.919,3953.611,4124.405,4542.760,5287.721,5395.101,5353.303,6396.166,6870.210,8299.278,9598.191,10386.312,11211.431,11937.672,12713.803,13543.306
2,Algeria,2462.085,2502.130,2446.588,2514.256,2640.191,2988.545,2929.092,2935.816,2326.571,2309.340,2684.465,1975.201,2033.196,2056.556,1674.567,1626.997,1783.410,1800.196,1772.368,1769.091,1947.755,1919.410,1958.238,2304.603,2839.820,3258.153,3690.572,4208.244,5214.762,4262.144,4941.493,5946.345,6057.974,5997.896,6108.878,4691.686,4426.580,4554.150,4568.244,4452.856,3757.585,4169.984,4981.822,5221.813,5579.128,5592.852,5645.511,5680.587,5705.771,5733.606
3,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49030.229,51957.313,45713.970,45630.195,46299.282,38877.485,39595.317,40017.741,42229.903,40688.491,36973.845,41806.876,41378.881,43809.726,45278.520,45994.879,46819.401,47330.444,47873.433,48458.183
4,Angola,1055.339,961.721,936.935,951.500,982.624,1074.327,977.905,1089.960,1151.190,1303.410,1397.414,1310.218,977.930,687.658,484.971,585.668,730.731,830.042,680.922,623.486,771.405,730.652,988.318,1122.365,1438.272,2128.257,2909.070,3493.014,4554.334,3630.103,4089.461,5175.442,5699.986,5880.641,6061.865,4680.829,3936.361,4628.964,3651.304,2926.143,1989.939,2445.388,3974.666,2967.384,2961.340,2990.597,3030.019,3140.940,3222.419,3316.369


In [94]:
gdp["Country"] = gdp["GDP per capita, current prices\n (U.S. dollars per capita)"]
gdp = gdp[["Country", 2023]]
gdp.rename(columns = {2023: "GDP_per_capita"}, inplace = True)
gdp.head()

,Country,GDP_per_capita
0,Afghanistan,410.933
1,Albania,8299.278
2,Algeria,5221.813
3,Andorra,43809.726
4,Angola,2967.384


In [95]:
df_applicants = df_applicants.merge(gdp.add_prefix("countries_"), how = "left", left_on = "countries_name", right_on = "countries_Country"
).drop(columns = ["countries_Country"])

In [96]:
print("Number of missing GDP's:", df_applicants["countries_GDP_per_capita"].isna().sum())

df_applicants[df_applicants["countries_GDP_per_capita"].isna()]["countries_name"].unique()

Number of missing GDP's: 552


array(['United States of America', 'Iran, Islamic Republic of',
       'Netherlands, Kingdom of the', 'Taiwan, Province of China',
       'China', 'United Kingdom of Great Britain and Northern Ireland',
       'North Macedonia', 'Türkiye', 'Slovakia',
       'Tanzania, United Republic of', 'Sao Tome and Principe', 'Czechia',
       'Hong Kong', 'Viet Nam', 'Bolivia, Plurinational State of',
       'Macao', 'Venezuela, Bolivarian Republic of', 'Niue',
       'Syrian Arab Republic', 'Sri Lanka', 'Gambia',
       'Northern Mariana Islands', 'Kyrgyzstan'], dtype=object)

There are some missing values that arose from the fact that some country names did not have a direct correspondence in the merge of the two datasets. These were filled manually using the imported Excel file as a source. For the remaining cases, the sources for filling the missing values were the following:
* Niue - https://www.cia.gov/the-world-factbook/countries/niue/factsheets/ (value from 2021)
* Syrian Arab Republic - https://tradingeconomics.com/syria/gdp-per-capita (value from 2021)
* Sri Lanka - https://tradingeconomics.com/sri-lanka/gdp-per-capita-us-dollar-wb-data.html
* Northern Mariana Islands - https://tradingeconomics.com/northern-mariana-islands/gdp-per-capita-current-us$-wb-data.html (value from 2022)

In [97]:
df_applicants.loc[df_applicants["countries_name"] == "Tanzania, United Republic of", "countries_GDP_per_capita"] = 1249.348
df_applicants.loc[df_applicants["countries_name"] == "Iran, Islamic Republic of", "countries_GDP_per_capita"] = 4347.193
df_applicants.loc[df_applicants["countries_name"] == "United States of America", "countries_GDP_per_capita"] = 82715.1
df_applicants.loc[df_applicants["countries_name"] == "Netherlands, Kingdom of the", "countries_GDP_per_capita"] = 64829.341
df_applicants.loc[df_applicants["countries_name"] == "Taiwan, Province of China", "countries_GDP_per_capita"] = 32404.32
df_applicants.loc[df_applicants["countries_name"] == "Czechia", "countries_GDP_per_capita"] = 31630.255
df_applicants.loc[df_applicants["countries_name"] == "China", "countries_GDP_per_capita"] = 12597.307
df_applicants.loc[df_applicants["countries_name"] == "Türkiye", "countries_GDP_per_capita"] = 13235.882
df_applicants.loc[df_applicants["countries_name"] == "United Kingdom of Great Britain and Northern Ireland", "countries_GDP_per_capita"] = 49647.559
df_applicants.loc[df_applicants["countries_name"] == "North Macedonia", "countries_GDP_per_capita"] = 8063.108
df_applicants.loc[df_applicants["countries_name"] == "Slovakia", "countries_GDP_per_capita"] = 24468.025
df_applicants.loc[df_applicants["countries_name"] == "Sao Tome and Principe", "countries_GDP_per_capita"] = 2949.403
df_applicants.loc[df_applicants["countries_name"] == "Hong Kong", "countries_GDP_per_capita"] = 50586.78
df_applicants.loc[df_applicants["countries_name"] == "Viet Nam", "countries_GDP_per_capita"] = 4324.049
df_applicants.loc[df_applicants["countries_name"] == "Venezuela, Bolivarian Republic of", "countries_GDP_per_capita"] = 3737.81
df_applicants.loc[df_applicants["countries_name"] == "Bolivia, Plurinational State of", "countries_GDP_per_capita"] = 3748.41
df_applicants.loc[df_applicants["countries_name"] == "Macao", "countries_GDP_per_capita"] = 69079.723
df_applicants.loc[df_applicants["countries_name"] == "Niue", "countries_GDP_per_capita"] = 11100
df_applicants.loc[df_applicants["countries_name"] == "Syrian Arab Republic", "countries_GDP_per_capita"] = 744.67
df_applicants.loc[df_applicants["countries_name"] == "Sri Lanka", "countries_GDP_per_capita"] = 3828
df_applicants.loc[df_applicants["countries_name"] == "Gambia", "countries_GDP_per_capita"] = 892.989
df_applicants.loc[df_applicants["countries_name"] == "Northern Mariana Islands", "countries_GDP_per_capita"] = 23786
df_applicants.loc[df_applicants["countries_name"] == "Kyrgyzstan", "countries_GDP_per_capita"] = 2018.911

Before moving on to another subset of variables, let's create a binary variable to flag students that are from Portugal.

In [98]:
df_applicants["is_Portuguese"] = (df_applicants["cd_pais"] == "PT").astype(int)
df_applicants["is_Portuguese"].value_counts()

is_Portuguese
1    9638
0    4027
Name: count, dtype: int64

#### Application Date and Time

Although we already have a hint (from the exploration stage) that these attributes have little to no predictive power, we will leave our final conclusion to the feature selection step.

In [99]:
df_applicants["datacandidaturafim_month"] = df_applicants["datacandidaturafim"].dt.month
df_applicants["datacandidaturafim_day"] = df_applicants["datacandidaturafim"].dt.day
df_applicants["datacandidaturafim_time_of_day"] = df_applicants["datacandidaturafim"].dt.hour.apply(tf.time_of_day)
df_applicants["datacandidaturafim_day_of_week"] = df_applicants["datacandidaturafim"].dt.day_name()

#### Program Names

As we saw earlier, there can be a varied number of ways in which the programs are presented by the applicants, translating to a considerable number of combinations of the `dprograma`, `dcurso`, and `DCursoOpcao` attributes. We could make this more uniform by aligning it with the data presented in our df_students table, which is displayed in a much cleaner fashion.

In [100]:
with pd.option_context("display.max_rows", None):
    display(df_applicants.groupby(["dprograma", "dcurso"])["DCursoOpcao"].value_counts(dropna = False).to_frame())

count
dprograma      dcurso                                                              DCursoOpcao                                                                                          
 Postgraduate  Postgraduate program                                                Enterprise Data Science & Analytics                                                               147
                                                                                   Digital Marketing and Analytics                                                                   133
                                                                                   Digital Enterprise Management                                                                     110
                                                                                   Data Science for Finance                                                                           50
                                                                                   Business Analytics for Hospitality & Tourism                                                       48
                                                                                   Marketing Intelligence                                                                             36
                                                                                   Data Science for Marketing                                                                         34
                                                                                   Business Intelligence and Analytics for Hospitality & Tourism                                      30
                                                                                   Information Systems Management                                                                     28
                                                                                   Statistical Systems - Specialization in Central Banks Statistics                                   28
                                                                                   Risk Analysis and Management                                                                       25
                                                                                   Information Analysis and Management                                                                24
                                                                                   Business Intelligence                                                                              23
                                                                                   Marketing Research and CRM                                                                         19
                                                                                   Knowledge Management and Business Intelligence                                                     18
                                                                                   Enterprise Information Systems                                                                     17
                                                                                   Information Management and Business Intelligence in Healthcare                                     17
                                                                                   Information Systems and Technologies Management                                                    17
                                                                                   Digital Transformation                                                                             14
                                                                                   Intelligence Management and Security                                                               10
                                                                                   Data Analytics            

First, we can transform `dprograma` by converting all sorts of typing variations into the same values. Then, we can combine `dcurso` and `DCursoOpcao` into a single column, and match it with the values in the `nm_curso_pt` column of df_students.

In [101]:
df_applicants["dprograma"] = df_applicants["dprograma"].apply(lambda x: "Postgraduate" if str(x).strip().startswith("P") else "Master")

df_applicants["dcurso"] = df_applicants["dcurso"].replace({"Pós-Graduação":
                                                                "Postgraduate program",
                                                           "Mestrado em Gestão de Informação":
                                                                "Master degree program in Information Management",
                                                           "Mestrado em Data Science and Advanced Analytics":
                                                                "Master degree program in Data Science and Advanced Analytics",
                                                            "Mestrado em Advanced Analytics":
                                                                "Master degree program in Data Science and Advanced Analytics",
                                                            "Mestrado em Data Driven Marketing":
                                                                "Master in Data Driven Marketing",
                                                            "Mestrado em Estatística e Gestão de Informação":
                                                                "Master degree program in Statistics and Information Management",
                                                            "Mestrado em Ciência e Sistemas de Informação Geográfica":
                                                                "Master degree program in Geographic Information Systems and Science",
                                                            "Mestrado em Direito e Mercados Financeiros":
                                                                "Master Degree in Law and Financial Markets"})

with pd.option_context("display.max_rows", None):
    display(df_applicants.groupby("dcurso")["DCursoOpcao"].value_counts(dropna = False).to_frame())

count
dcurso                                                              DCursoOpcao                                                                                          
European Master of Science in Information Systems Management        European Master of Science in Information Systems Management                                      109
Master Degree in Law and Financial Markets                          Mestrado em Direito e Mercados Financeiros                                                         59
                                                                    Master Degree in Law and Financial Markets                                                          2
Master degree program in Data Science and Advanced Analytics        especialização em Business Analytics                                                              545
                                                                    especialização em Data Science                                                                    527
                                                                    specialization in Data Science                                                                    335
                                                                    Major in Data Science                                                                             277
                                                                    specialization in Business Analytics                                                              268
                                                                    Major in Business Analytics                                                                       227
                                                                    Data Science and Advanced Analytics                                                               141
                                                                    Major em Data Science                                                                              45
                                                                    Major em Business Analytics                                                                        41
                                                                    Especialização em Data Science                                                                      5
                                                                    Especialização Business Analytics                                                                   4
                                                                    Advanced Analytics                                                                                  3
                                                                    Specialization Business Analytics                                                                   3
                                                                    Specialization Data Science                                                                         3
Master degree program in Geographic Information Systems and Science Mestrado em Ciência e Sistemas de Informação Geográfica (Regime e-Learning)                        74
                                                                    especialização em Ciência e Sistemas de Informação Geográfica (Regime e-Learning)                  44
                                                                    especialização em Ciência dos Dados Geospaciais (Regime e-Learning)                                31
                                                                    especialização em Ciência e Sistemas de Informação Geográfica (Regime semi-presencial)             16
                                                                    Mestrado em Ciência e Sistemas de Informação Geográfica (Regime semi-presencial)                   15
                                                                    especialização e

In this process, we can also take the chance to create two additional columns: `is_After_Working_Hours` will be 1 if the program takes place after working hours, while `is_Fully_In_Person` will be 0 if the program is taken in some form of blended or distance-learning. We will assume the cases where this information is omitted to be taken during working hours in a fully-presencial regime.

In [102]:
df_applicants["is_After_Working_Hours"] = df_applicants["DCursoOpcao"].str.contains(
    "After Working Hours|Laboral", case = False, na = False).astype(int)

df_applicants["is_Fully_In_Person"] = df_applicants["DCursoOpcao"].str.contains(
    "e-Learning|b-Learning|semi-presencial|semi-presence|semi presencial", case = False, na = False).astype(int).apply(lambda x: 0 if x == 1 else 1)

We will now proceed to cleaning the program names. We will ignore specializations, as well as other details such as being online of after working hours. We will first create a column, `Program_Name` that aggregates `dcurso` and `DCursoOpcao` into a single attribute, and then enforce a set of rules to make it consistent with the `nm_curso_pt` values shown below.

In [103]:
sorted(df_students["nm_curso_pt"].unique().tolist())

['Master of Science in Geospatial Technologies',
 'Mestrado em Ciência de Dados e Métodos Analíticos Avançados',
 'Mestrado em Ciência e Sistemas de Informação Geográfica',
 'Mestrado em Direito e Mercados Financeiros',
 'Mestrado em Estatística e Gestão de Informação',
 'Mestrado em Gestão de Informação',
 'Mestrado em Marketing Analítico (Data Driven Marketing)',
 'Post-graduate program in Business Intelligence & Analytics for Hospitality and Tourism',
 'Pós-Graduação em Análise e Gestão de Informação',
 'Pós-Graduação em Análise e Gestão de Risco',
 'Pós-Graduação em Business Intelligence',
 'Pós-Graduação em Cidades Inteligentes (Smart Cities)',
 'Pós-Graduação em Ciência e Sistemas de Informação Geográfica',
 'Pós-Graduação em Data Science for Finance',
 'Pós-Graduação em Data Science for Marketing',
 'Pós-Graduação em Digital Enterprise Management',
 'Pós-Graduação em Digital Innovation for Business',
 'Pós-Graduação em Digital Marketing and Analytics',
 'Pós-Graduação em Enterpr

In [104]:
df_applicants["Program_Name"] = df_applicants["dcurso"] + " - " + df_applicants["DCursoOpcao"]
sorted(df_applicants["Program_Name"].unique().tolist())

['European Master of Science in Information Systems Management - European Master of Science in Information Systems Management',
 'Master Degree in Law and Financial Markets - Master Degree in Law and Financial Markets',
 'Master Degree in Law and Financial Markets - Mestrado em Direito e Mercados Financeiros',
 'Master degree program in Data Science and Advanced Analytics - Advanced Analytics',
 'Master degree program in Data Science and Advanced Analytics - Data Science and Advanced Analytics',
 'Master degree program in Data Science and Advanced Analytics - Especialização Business Analytics',
 'Master degree program in Data Science and Advanced Analytics - Especialização em Data Science',
 'Master degree program in Data Science and Advanced Analytics - Major em Business Analytics',
 'Master degree program in Data Science and Advanced Analytics - Major em Data Science',
 'Master degree program in Data Science and Advanced Analytics - Major in Business Analytics',
 'Master degree progr

This function will help us perform this burdensome task (in case we need to repeat it at some point in the future).

In [105]:
df_applicants["Program_Name"] = df_applicants["Program_Name"].apply(tf.transform_program_name)
df_applicants.head()

,idCandidatura,DGrauAcademicoPT_1,DGrauAcademicoPT_2,DGrauAcademicoPT_3,DEstabEnsinoProvPT_1,DEstabEnsinoProvPT_2,DEstabEnsinoProvPT_3,DCursoProvPT_1,DCursoProvPT_2,DCursoProvPT_3,CursoConcluido_1,CursoConcluido_2,CursoConcluido_3,NotaOuMedia_1,NotaOuMedia_2,NotaOuMedia_3,DescEscala_1,DescEscala_2,DescEscala_3,ID_INDIVIDUO,idcandidato,idgenero,cd_pais,Localidade,nivelInglesCompreensao,nivelInglesFala,nivelInglesEscrita,datacandidaturafim,dprograma,dcurso,DCursoOpcao,idEstadoCandidatura,DEstadoPT,NumberAcademicExperiences,NumberAcademicDegrees,HighestAcademicDegree,Country_1,Country_2,Country_3,Rank,Student Population,Students to Staff Ratio,Overall Score,Teaching,Research Environment,Research Quality,Industry Impact,International Outlook,Percentage_International_Students,Percentage_Female_Students,Is_Ranked,Previous_IMS_Student,Previous_Data_Studies,Previous_Computer_Studies,Previous_Tech_Studies,Previous_Math_Studies,Previous_Engineering_Studies,Previous_Finance_Studies,Previous_Marketing_Studies,Previous_Field_Studies,Number_Studied_Fields,Non_Finished_Programs,Prop_Finished_Programs,MinimoEscala_1,MaximoEscala_1,MinimoEscala_2,MaximoEscala_2,MinimoEscala_3,MaximoEscala_3,ScaledGrade_1,ScaledGrade_2,ScaledGrade_3,AverageScaledGrade,MaxScaledGrade,MinScaledGrade,countries_name,countries_alpha-3,countries_region,countries_sub-region,countries_dist,countries_GDP_per_capita,is_Portuguese,datacandidaturafim_month,datacandidaturafim_day,datacandidaturafim_time_of_day,datacandidaturafim_day_of_week,is_After_Working_Hours,is_Fully_In_Person,Program_Name
0,6731,Licenciado,None,None,Delhi university,None,None,BA Pass,None,None,S,None,None,45.0,NaN,NaN,Minimal positive: 35% | Maximum: 100%,None,None,NaN,8833,M,PT,amadora,Fluente (QERC: C2),Fluente (QERC: C2),Fluente (QERC: C2),2018-01-08 16:23:42.327,Postgraduate,Postgraduate program,Sistemas de Informação Empresariais,4,Completa,1,1,Licenciado,United States,NaN,NaN,1758.0,7044.0,12.1,20.9800,24.1,7.7,28.4,17.0,32.5,6.0,71.0,1,0,0,0,0,0,0,0,0,0,0,0,1.0,35.0,100.0,NaN,NaN,NaN,NaN,0.45,NaN,NaN,0.45,0.45,0.45,Portugal,PRT,Europe,Southern Europe,114.3257,27834.773,1,1,8,Afternoon,Monday,0,1,Pós-Graduação em Sistemas de Informação Empresariais
1,6739,Licenciado,None,None,Universidade Técnica de Lisboa - Instituto Superior de Economia e Gestão,None,None,Economia,None,None,S,None,None,12.0,NaN,NaN,Minímo positivo: 10 | Máximo: 20,None,None,14404.0,6719,M,PT,Lisboa,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),2018-01-09 22:09:18.147,Postgraduate,Postgraduate program,Sistemas de Informação Empresariais,6,Admitido,1,1,Licenciado,NaN,NaN,NaN,2100.0,14080.0,14.8,32.5875,24.3,16.1,53.3,36.0,54.8,16.0,55.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,10.0,20.0,NaN,NaN,NaN,NaN,0.60,NaN,NaN,0.60,0.60,0.60,Portugal,PRT,Europe,Southern Europe,114.3257,27834.773,1,1,9,Evening,Tuesday,0,1,Pós-Graduação em Sistemas de Informação Empresariais
2,6766,Mestre,None,None,Universidade Lusíada,None,None,Arquitectura,None,None,S,None,None,14.0,NaN,NaN,Minímo positivo: 10 | Máximo: 20,None,None,NaN,8408,F,PT,Vila Franca de Xira,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),2018-01-15 21:14:12.810,Postgraduate,Postgraduate program,Sistemas de Informação Empresariais,4,Completa,1,1,Mestre,Portugal,NaN,NaN,1550.0,11773.0,9.7,24.4525,24.3,9.7,31.3,20.9,56.6,17.0,55.0,1,0,0,0,1,0,0,0,0,1,1,0,1.0,10.0,20.0,NaN,NaN,NaN,NaN,0.70,NaN,NaN,0.70,0.70,0.70,Portugal,PRT,Europe,Southern Europe,114.3257,27834.773,1,1,15,Evening,Monday,0,1,Pós-Graduação em Sistemas de Informação Empresariais
3,6773,Licenciado,Mestre,None,Universidade de Lisboa,Universidade de Lisboa,None,Planeamento,Sistemas de Informação Geográfica e Modelação Territorial Apl Ordenamento d,None,S,N,None,13.0,15.0,NaN,Minímo positivo: 10 | Máximo: 20,Minímo positivo: 10 | Máximo: 20,None,NaN,8945,M,PT,Alfeizerão,Nativo ou bilingue,Nativo ou bilingue,Nativo ou bilingue,2018-01-19 13:04:12.207,Postgraduate,Postgraduate program,Inteligência Geoespacial,6,Ad

Now that we added these new variables of interest regarding the applications' data, let's proceed to the information regarding the candidates' past and current professional background.

#### Bringing In Professional Activity-Related Variables

We will start by merging the df_applicants and df_profactiv datasets, so that we begin to aggregate information in a single table.

In [106]:
df_applicants_profactiv = pd.merge(df_applicants, df_profactiv, how = "left",
                                   left_on = "idcandidato", right_on = "idCandidato").drop(columns=["idCandidato"])
df_applicants_profactiv.head()

,idCandidatura,DGrauAcademicoPT_1,DGrauAcademicoPT_2,DGrauAcademicoPT_3,DEstabEnsinoProvPT_1,DEstabEnsinoProvPT_2,DEstabEnsinoProvPT_3,DCursoProvPT_1,DCursoProvPT_2,DCursoProvPT_3,CursoConcluido_1,CursoConcluido_2,CursoConcluido_3,NotaOuMedia_1,NotaOuMedia_2,NotaOuMedia_3,DescEscala_1,DescEscala_2,DescEscala_3,ID_INDIVIDUO,idcandidato,idgenero,cd_pais,Localidade,nivelInglesCompreensao,nivelInglesFala,nivelInglesEscrita,datacandidaturafim,dprograma,dcurso,DCursoOpcao,idEstadoCandidatura,DEstadoPT,NumberAcademicExperiences,NumberAcademicDegrees,HighestAcademicDegree,Country_1,Country_2,Country_3,Rank,Student Population,Students to Staff Ratio,Overall Score,Teaching,Research Environment,Research Quality,Industry Impact,International Outlook,Percentage_International_Students,Percentage_Female_Students,Is_Ranked,Previous_IMS_Student,Previous_Data_Studies,Previous_Computer_Studies,Previous_Tech_Studies,Previous_Math_Studies,Previous_Engineering_Studies,Previous_Finance_Studies,Previous_Marketing_Studies,Previous_Field_Studies,Number_Studied_Fields,Non_Finished_Programs,Prop_Finished_Programs,MinimoEscala_1,MaximoEscala_1,MinimoEscala_2,MaximoEscala_2,MinimoEscala_3,MaximoEscala_3,ScaledGrade_1,ScaledGrade_2,ScaledGrade_3,AverageScaledGrade,MaxScaledGrade,MinScaledGrade,countries_name,countries_alpha-3,countries_region,countries_sub-region,countries_dist,countries_GDP_per_capita,is_Portuguese,datacandidaturafim_month,datacandidaturafim_day,datacandidaturafim_time_of_day,datacandidaturafim_day_of_week,is_After_Working_Hours,is_Fully_In_Person,Program_Name,DataInicio_1,DataInicio_2,DataInicio_3,DataInicio_4,DataInicio_5,DataInicio_6,DataFim_1,DataFim_2,DataFim_3,DataFim_4,DataFim_5,DataFim_6,ActivProEntidadePatronal_1,ActivProEntidadePatronal_2,ActivProEntidadePatronal_3,ActivProEntidadePatronal_4,ActivProEntidadePatronal_5,ActivProEntidadePatronal_6,ActivProFuncao_1,ActivProFuncao_2,ActivProFuncao_3,ActivProFuncao_4,ActivProFuncao_5,ActivProFuncao_6
0,6731,Licenciado,None,None,Delhi university,None,None,BA Pass,None,None,S,None,None,45.0,NaN,NaN,Minimal positive: 35% | Maximum: 100%,None,None,NaN,8833,M,PT,amadora,Fluente (QERC: C2),Fluente (QERC: C2),Fluente (QERC: C2),2018-01-08 16:23:42.327,Postgraduate,Postgraduate program,Sistemas de Informação Empresariais,4,Completa,1,1,Licenciado,United States,NaN,NaN,1758.0,7044.0,12.1,20.9800,24.1,7.7,28.4,17.0,32.5,6.0,71.0,1,0,0,0,0,0,0,0,0,0,0,0,1.0,35.0,100.0,NaN,NaN,NaN,NaN,0.45,NaN,NaN,0.45,0.45,0.45,Portugal,PRT,Europe,Southern Europe,114.3257,27834.773,1,1,8,Afternoon,Monday,0,1,Pós-Graduação em Sistemas de Informação Empresariais,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Accenture,None,None,None,None,None,IT Production Support Sr. Analyst,None,None,None,None,None
1,6739,Licenciado,None,None,Universidade Técnica de Lisboa - Instituto Superior de Economia e Gestão,None,None,Economia,None,None,S,None,None,12.0,NaN,NaN,Minímo positivo: 10 | Máximo: 20,None,None,14404.0,6719,M,PT,Lisboa,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),2018-01-09 22:09:18.147,Postgraduate,Postgraduate program,Sistemas de Informação Empresariais,6,Admitido,1,1,Licenciado,NaN,NaN,NaN,2100.0,14080.0,14.8,32.5875,24.3,16.1,53.3,36.0,54.8,16.0,55.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,10.0,20.0,NaN,NaN,NaN,NaN,0.60,NaN,NaN,0.60,0.60,0.60,Portugal,PRT,Europe,Southern Europe,114.3257,27834.773,1,1,9,Evening,Tuesday,0,1,Pós-Graduação em Sistemas de Informação Empresariais,2017-10-01,2018-05-15,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,AMT Consulting,James (formerly CrowdProcess),None,None,None,None,Consultor de SAP,Junior Credit Risk Data Scientist,None,None,None,None
2,6766,Mestre,None,None,Universidade Lusíada,None,None,Arquitectura,None,None,S,None,None,14.0,NaN,NaN,Minímo positivo: 10 | Máximo: 20,None,None,NaN,8408,F,PT,Vila Franca de Xira,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),2018-01-15 21:14:12.810,Postgraduate,Postgraduate program,Sistemas de Informação Empresariais,4,Completa,

#### Number of Professional Experiences

The first variable that we are going to bring will simply count the number of past and present professional experiences taken by the candidate at the time of their application. In the exploration stage, we learned that it is frequent for applicants to omit at least one of the fields that describe their experience (start date, ending date, employer and job description). Therefore, we will consider the presence of a single field (for each of the 6 levels as a sign that the individual underwent a professional endeavour.

In [107]:
df_profactiv.head()

,idCandidato,DataInicio_1,DataInicio_2,DataInicio_3,DataInicio_4,DataInicio_5,DataInicio_6,DataFim_1,DataFim_2,DataFim_3,DataFim_4,DataFim_5,DataFim_6,ActivProEntidadePatronal_1,ActivProEntidadePatronal_2,ActivProEntidadePatronal_3,ActivProEntidadePatronal_4,ActivProEntidadePatronal_5,ActivProEntidadePatronal_6,ActivProFuncao_1,ActivProFuncao_2,ActivProFuncao_3,ActivProFuncao_4,ActivProFuncao_5,ActivProFuncao_6
0,121,2008-09-01,2018-09-01,2021-01-01,NaT,NaT,NaT,NaT,2020-12-31,NaT,NaT,NaT,NaT,"AEG Telecomunicações, SA",NaN,OutSystems,None,None,None,Responsável dos Sistemas de Informação,Customer Success Analytics Team Lead,Head Data & Analytics for R&D,None,None,None
1,182,2010-04-01,2014-06-01,2016-05-16,NaT,NaT,NaT,2014-05-31,2016-05-15,NaT,NaT,NaT,NaT,agap2IT Portugal,Closer Consulting,Linkare TI - Tecnologias de Informação,None,None,None,Business Analyst & Project Manager,Business Analyst & Project Manager,Business Analyst & Project Manager,None,None,None
2,286,2018-02-01,2018-08-16,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Universidade Nova de Lisboa - Instituto Superior de Estatística e Gestão de Informação,Escola Profissional de Ciências Geográficas,None,None,None,None,Postgraduate Programs Manager,Técnica Especializada - Ensino de SIG,None,None,None,None
3,296,2016-02-01,2019-07-19,2024-10-17,NaT,NaT,NaT,2019-07-19,2024-10-17,NaT,NaT,NaT,NaT,BNP Paribas,CISCO,Any,None,None,None,Client Support - Team Coordinator,Installed Base Analyst - Team Lead,Unemployed,None,None,None
4,366,2017-09-01,2018-02-01,NaT,NaT,NaT,NaT,2018-01-30,NaT,NaT,NaT,NaT,NaT,Nova-IMS,Nova-IMS,None,None,None,None,Professor Convidado de Ciência e Sistemas de Informação Geogáfica,Professor Convidado de Location Based Services,None,None,None,None


In [108]:
prefixes = ["1", "2", "3", "4", "5", "6"]
variables = ["DataInicio", "DataFim", "ActivProEntidadePatronal", "ActivProFuncao"]

df_applicants_profactiv["Number_Professional_Experiences"] = df_applicants_profactiv.apply(
    lambda row: sum(any(pd.notna(row[f"{var}_{n}"]) for var in variables) for n in prefixes), axis = 1)

#### Years of Experience

In the previous notebook, dedicated to exploratory data analysis, we created the `Years_of_Experience` and `Years_of_Longest_Prof_Experience` variables. We will bring these fields once again to attempt to increase our predictive power.

In [109]:
start_date_columns = ["DataInicio_1", "DataInicio_2", "DataInicio_3", "DataInicio_4", "DataInicio_5", "DataInicio_6"]

df_applicants_profactiv["Years_of_Experience"] = (
    df_applicants_profactiv["datacandidaturafim"].dt.year - df_applicants_profactiv[start_date_columns].min(axis = 1).dt.year
).fillna(0).clip(lower = 0).astype(int)

In [110]:
end_date_columns = ["DataFim_1", "DataFim_2", "DataFim_3", "DataFim_4", "DataFim_5", "DataFim_6"]

durations = []
for inicio_col, fim_col in zip(start_date_columns, end_date_columns):
    start_year = df_applicants_profactiv[inicio_col].dt.year
    end_year = df_applicants_profactiv[fim_col].dt.year.fillna(df_applicants_profactiv["datacandidaturafim"].dt.year)
    duration = (end_year - start_year).fillna(0).clip(lower = 0)
    durations.append(duration)

df_applicants_profactiv["Years_of_Longest_Prof_Experience"] = pd.concat(durations, axis = 1).max(axis = 1)

#### Working Upon Application

We are also interested in understanding if a candidate that was working at the time of the application is more or less likely to be admitted, and how this fact impacts their academic performance.

In [111]:
df_applicants_profactiv["Working_Upon_Application"] = df_applicants_profactiv.apply(
    lambda row: tf.working_upon_application(
        row = row, start_date_columns = start_date_columns, end_date_columns = end_date_columns, positive_value = 1, negative_value = 0), 
    axis = 1)

#### Previous Jobs

As we did for the study fiels, we will also explore the possibility that applicants with a professional background in certain specified fields could be more (or less) likely to be admitted and have a better (or worse academic performance). For that purpose, we will create new attributes using the `ActivProFuncao` variables as the basis of our transformation.

In [112]:
working_fields = {
    "Previous_Data_Jobs": ["Data", "Dados", "Informação", "Information"],
    "Previous_Analytics_Jobs": ["Análise", "Analise", "Analista", "Analytics", "Analyst"],
    "Previous_Computing_Jobs": ["Comput", "Software", "Informática", "Informatics", "Dev"],
    "Previous_Tech_Jobs": ["Tech", "Tecno", "IT"],
    "Previous_Math_Jobs": ["Math", "Mat", "Estat", "Stat"],
    "Previous_Engineering_Jobs": ["Engenh", "Engine"],
    "Previous_Finance_Jobs": ["Finan"],
    "Previous_Marketing_Jobs": ["Marketing", "Mkt"],
    "Previous_Project_Jobs": ["Proj"]}

for column_name, terms in working_fields.items():
    df_applicants_profactiv = tf.create_conditional_column(
        df = df_applicants_profactiv,
        columns_to_find = ["ActivProFuncao_1", "ActivProFuncao_2", "ActivProFuncao_3", "ActivProFuncao_4", "ActivProFuncao_5", "ActivProFuncao_6"],
        new_column = column_name,
        values_to_find = terms,
        positive_value = 1,
        negative_value = 0)

As we did earlier, we will again aggregate this information in two new columns: `Previous_Field_Jobs` will be 1 if the applicant worked in any of the mentioned fields above, while `Number_Fields_Worked` will sum the number of fields in which the applicant worked. Once again, we expect these variables to be strongly correlated, but we will leave the decision on which to keep to the feature selection stage.

In [113]:
df_applicants_profactiv["Previous_Field_Jobs"] = df_applicants_profactiv[list(working_fields.keys())].any(axis=1).astype(int)
df_applicants_profactiv["Number_Fields_Worked"] = df_applicants_profactiv[list(working_fields.keys())].sum(axis=1)

#### Previous Internships (and summer jobs)

We saw that applicants that undertook an internship or a summer job prior to their application could potentially have a higher likelihood of acceptance when applying to postgraduate or masters' programs at Nova IMS. Therefore, let's bring this attribute to our data.

In [114]:
df_applicants_profactiv = tf.create_conditional_column(
    df = df_applicants_profactiv,
    columns_to_find = ["ActivProFuncao_1", "ActivProFuncao_2", "ActivProFuncao_3", "ActivProFuncao_4", "ActivProFuncao_5", "ActivProFuncao_6"],
    new_column = "Previous_Trainee_Jobs",
    values_to_find = ["Intern", "Trainee", "Estag", "Summer", "Verão"],
    positive_value = 1,
    negative_value = 0)

#### Previous Job at Nova IMS' Partners

Similarly, we learned that having a professional background at a partner institution of Nova IMS could impact the applicants' admission and academic performance. Once again, we will base the creation of this variable on the [AD Nova IMS](https://adnova.novaims.unl.pt/en/companies/partnerships/) website, as well as the masters' programs officical pages (available from [here](https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/)).

In [115]:
df_applicants_profactiv = tf.create_conditional_column(
    df = df_applicants_profactiv,
    columns_to_find = ["ActivProEntidadePatronal_1", "ActivProEntidadePatronal_2", "ActivProEntidadePatronal_3",
                       "ActivProEntidadePatronal_4", "ActivProEntidadePatronal_5", "ActivProEntidadePatronal_6"],
    new_column = "Previous_Jobs_in_Partners",
    values_to_find = ["Confederação do Turismo de Portugal", "Turismo de Portugal", "HURB", "CLEVER", "NEST", "Vila Galé", "UNIGIS",
                      "Google", "IDC", "SAP", "NOVA School of Law", "Microsoft", "Academia Militar", "Esri Portugal",
                      "Centro de Informação Geoespacial do Exército", "United States Geospatial Intelligence Foundation",
                      "Faculdade de Ciências Médicas - NOVA", "Escola Nacional de Saúde Pública - NOVA",
                      "Instituto de Higiene e Medicina Tropical - NOVA", "Instituto da Defesa Nacional",
                      "Sistema de Informações da Républica Portuguesa", "ENSP-NOVA", "NOVA Medical School", "Universidade de Aveiro",
                      "Inspeção-Geral de Finanças", "ISCTE - Executive Education", "Irving Fisher Committee on Central Bank Statistics",
                      "Banco de Portugal", "European Central Bank", "European Master in Official Statistics", "INE", "IAPMEI", "EDP",
                      "Accenture", "Fidelidade", "SAS", "Portugal Oeste", "Jerónimo Martins", "Tranquilidade", "Future Healthcare", "iFood",
                      "BI4ALL", "Ageas", "Feedzai"],
    positive_value = 1,
    negative_value = 0)

#### Bringing In Student and Grades-Related Variables

Finally, we will bring the variables that concern only the applicants that were admitted to Nova IMS, namely their final GPA.

Earlier, we understood that the merge of the datasets may not have been 100% correctly executed, as there appeared to be students whose final grades were related to a different program than the one mentioned on their application. Therefore, and since we already had the work of cleaning the program names to match `nm_curso_pt` in df_students, we will add this as a criterium for the merge to happen.

In [116]:
df_full = pd.merge(df_applicants_profactiv, df_students, how = "left",
                   left_on = ["ID_INDIVIDUO", "Program_Name"], right_on = ["id_individuo", "nm_curso_pt"]).drop(columns = ["id_individuo",
                                                                                                                           "nm_curso_pt"])
df_full.head()

,idCandidatura,DGrauAcademicoPT_1,DGrauAcademicoPT_2,DGrauAcademicoPT_3,DEstabEnsinoProvPT_1,DEstabEnsinoProvPT_2,DEstabEnsinoProvPT_3,DCursoProvPT_1,DCursoProvPT_2,DCursoProvPT_3,CursoConcluido_1,CursoConcluido_2,CursoConcluido_3,NotaOuMedia_1,NotaOuMedia_2,NotaOuMedia_3,DescEscala_1,DescEscala_2,DescEscala_3,ID_INDIVIDUO,idcandidato,idgenero,cd_pais,Localidade,nivelInglesCompreensao,nivelInglesFala,nivelInglesEscrita,datacandidaturafim,dprograma,dcurso,DCursoOpcao,idEstadoCandidatura,DEstadoPT,NumberAcademicExperiences,NumberAcademicDegrees,HighestAcademicDegree,Country_1,Country_2,Country_3,Rank,Student Population,Students to Staff Ratio,Overall Score,Teaching,Research Environment,Research Quality,Industry Impact,International Outlook,Percentage_International_Students,Percentage_Female_Students,Is_Ranked,Previous_IMS_Student,Previous_Data_Studies,Previous_Computer_Studies,Previous_Tech_Studies,Previous_Math_Studies,Previous_Engineering_Studies,Previous_Finance_Studies,Previous_Marketing_Studies,Previous_Field_Studies,Number_Studied_Fields,Non_Finished_Programs,Prop_Finished_Programs,MinimoEscala_1,MaximoEscala_1,MinimoEscala_2,MaximoEscala_2,MinimoEscala_3,MaximoEscala_3,ScaledGrade_1,ScaledGrade_2,ScaledGrade_3,AverageScaledGrade,MaxScaledGrade,MinScaledGrade,countries_name,countries_alpha-3,countries_region,countries_sub-region,countries_dist,countries_GDP_per_capita,is_Portuguese,datacandidaturafim_month,datacandidaturafim_day,datacandidaturafim_time_of_day,datacandidaturafim_day_of_week,is_After_Working_Hours,is_Fully_In_Person,Program_Name,DataInicio_1,DataInicio_2,DataInicio_3,DataInicio_4,DataInicio_5,DataInicio_6,DataFim_1,DataFim_2,DataFim_3,DataFim_4,DataFim_5,DataFim_6,ActivProEntidadePatronal_1,ActivProEntidadePatronal_2,ActivProEntidadePatronal_3,ActivProEntidadePatronal_4,ActivProEntidadePatronal_5,ActivProEntidadePatronal_6,ActivProFuncao_1,ActivProFuncao_2,ActivProFuncao_3,ActivProFuncao_4,ActivProFuncao_5,ActivProFuncao_6,Number_Professional_Experiences,Years_of_Experience,Years_of_Longest_Prof_Experience,Working_Upon_Application,Previous_Data_Jobs,Previous_Analytics_Jobs,Previous_Computing_Jobs,Previous_Tech_Jobs,Previous_Math_Jobs,Previous_Engineering_Jobs,Previous_Finance_Jobs,Previous_Marketing_Jobs,Previous_Project_Jobs,Previous_Field_Jobs,Number_Fields_Worked,Previous_Trainee_Jobs,Previous_Jobs_in_Partners,DT_MATRIC,dataConclusao,NotaFinal,situacaoFinal,cd_curso
0,6731,Licenciado,None,None,Delhi university,None,None,BA Pass,None,None,S,None,None,45.0,NaN,NaN,Minimal positive: 35% | Maximum: 100%,None,None,NaN,8833,M,PT,amadora,Fluente (QERC: C2),Fluente (QERC: C2),Fluente (QERC: C2),2018-01-08 16:23:42.327,Postgraduate,Postgraduate program,Sistemas de Informação Empresariais,4,Completa,1,1,Licenciado,United States,NaN,NaN,1758.0,7044.0,12.1,20.9800,24.1,7.7,28.4,17.0,32.5,6.0,71.0,1,0,0,0,0,0,0,0,0,0,0,0,1.0,35.0,100.0,NaN,NaN,NaN,NaN,0.45,NaN,NaN,0.45,0.45,0.45,Portugal,PRT,Europe,Southern Europe,114.3257,27834.773,1,1,8,Afternoon,Monday,0,1,Pós-Graduação em Sistemas de Informação Empresariais,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Accenture,None,None,None,None,None,IT Production Support Sr. Analyst,None,None,None,None,None,1,0,0.0,0,0,1,0,1,0,0,0,0,0,1,2,0,1,NaT,NaT,NaN,NaN,NaN
1,6739,Licenciado,None,None,Universidade Técnica de Lisboa - Instituto Superior de Economia e Gestão,None,None,Economia,None,None,S,None,None,12.0,NaN,NaN,Minímo positivo: 10 | Máximo: 20,None,None,14404.0,6719,M,PT,Lisboa,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),2018-01-09 22:09:18.147,Postgraduate,Postgraduate program,Sistemas de Informação Empresariais,6,Admitido,1,1,Licenciado,NaN,NaN,NaN,2100.0,14080.0,14.8,32.5875,24.3,16.1,53.3,36.0,54.8,16.0,55.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,10.0,20.0,NaN,NaN,NaN,NaN,0.60,NaN,NaN,0.60,0.60,0.60,Portugal,PRT,Europe,Southern Europe,114.3257,27834.773,1,1,9,Evening,Tuesday,0,1,Pós-Graduação em Sistemas de Informação Empresariais,2017-10-01,2018-05-15,NaT,NaT,Na

Let's check how many rows we have with a final GPA (`NotaFinal`).

In [117]:
print("Number of Final GPA values:", df_full["NotaFinal"].notna().sum())
print("Percentage of Final GPA values not missing:", round(df_full["NotaFinal"].notna().sum() / df_full.shape[0], 2))

Number of Final GPA values: 3220
Percentage of Final GPA values not missing: 0.24


We still managed to keep around 3 200 students that are not missing a final GPA. Although we could desire to have a bigger sample, this will have to be enough to develop and test the grades predictive ability.

We should also be sure that this time, there are no students that finished the program (`dataConclusao`) before concluding their application (`datacandidaturafim`). This was one of the biggest hints that the merge had not been correctly performed, and therefore should be an issue that we could solve this time around.

In [118]:
df_full[df_full["dataConclusao"] < df_full["datacandidaturafim"]]

,idCandidatura,DGrauAcademicoPT_1,DGrauAcademicoPT_2,DGrauAcademicoPT_3,DEstabEnsinoProvPT_1,DEstabEnsinoProvPT_2,DEstabEnsinoProvPT_3,DCursoProvPT_1,DCursoProvPT_2,DCursoProvPT_3,CursoConcluido_1,CursoConcluido_2,CursoConcluido_3,NotaOuMedia_1,NotaOuMedia_2,NotaOuMedia_3,DescEscala_1,DescEscala_2,DescEscala_3,ID_INDIVIDUO,idcandidato,idgenero,cd_pais,Localidade,nivelInglesCompreensao,nivelInglesFala,nivelInglesEscrita,datacandidaturafim,dprograma,dcurso,DCursoOpcao,idEstadoCandidatura,DEstadoPT,NumberAcademicExperiences,NumberAcademicDegrees,HighestAcademicDegree,Country_1,Country_2,Country_3,Rank,Student Population,Students to Staff Ratio,Overall Score,Teaching,Research Environment,Research Quality,Industry Impact,International Outlook,Percentage_International_Students,Percentage_Female_Students,Is_Ranked,Previous_IMS_Student,Previous_Data_Studies,Previous_Computer_Studies,Previous_Tech_Studies,Previous_Math_Studies,Previous_Engineering_Studies,Previous_Finance_Studies,Previous_Marketing_Studies,Previous_Field_Studies,Number_Studied_Fields,Non_Finished_Programs,Prop_Finished_Programs,MinimoEscala_1,MaximoEscala_1,MinimoEscala_2,MaximoEscala_2,MinimoEscala_3,MaximoEscala_3,ScaledGrade_1,ScaledGrade_2,ScaledGrade_3,AverageScaledGrade,MaxScaledGrade,MinScaledGrade,countries_name,countries_alpha-3,countries_region,countries_sub-region,countries_dist,countries_GDP_per_capita,is_Portuguese,datacandidaturafim_month,datacandidaturafim_day,datacandidaturafim_time_of_day,datacandidaturafim_day_of_week,is_After_Working_Hours,is_Fully_In_Person,Program_Name,DataInicio_1,DataInicio_2,DataInicio_3,DataInicio_4,DataInicio_5,DataInicio_6,DataFim_1,DataFim_2,DataFim_3,DataFim_4,DataFim_5,DataFim_6,ActivProEntidadePatronal_1,ActivProEntidadePatronal_2,ActivProEntidadePatronal_3,ActivProEntidadePatronal_4,ActivProEntidadePatronal_5,ActivProEntidadePatronal_6,ActivProFuncao_1,ActivProFuncao_2,ActivProFuncao_3,ActivProFuncao_4,ActivProFuncao_5,ActivProFuncao_6,Number_Professional_Experiences,Years_of_Experience,Years_of_Longest_Prof_Experience,Working_Upon_Application,Previous_Data_Jobs,Previous_Analytics_Jobs,Previous_Computing_Jobs,Previous_Tech_Jobs,Previous_Math_Jobs,Previous_Engineering_Jobs,Previous_Finance_Jobs,Previous_Marketing_Jobs,Previous_Project_Jobs,Previous_Field_Jobs,Number_Fields_Worked,Previous_Trainee_Jobs,Previous_Jobs_in_Partners,DT_MATRIC,dataConclusao,NotaFinal,situacaoFinal,cd_curso


Amazing! Now, all student grades seem to be aligned with their respective program instances.

We have not finalized the process of adding new variables to our dataset (both from the existing data and from outside sources), while also joining the 3 tables we initially had into a single reference table.

Before proceeding to the next stage of our work, where we will encode the attributes that are in need of such transformation, we will drop the columns that are not going to be used later on, either because their data types are not supported, or they are variables of no interest to our future developments (since they are already encoded in some form, or are simply not relevant).

In [119]:
print("Number of columns in the dataset:", df_full.shape[1])

df_full.columns.to_list()

Number of columns in the dataset: 135


['idCandidatura',
 'DGrauAcademicoPT_1',
 'DGrauAcademicoPT_2',
 'DGrauAcademicoPT_3',
 'DEstabEnsinoProvPT_1',
 'DEstabEnsinoProvPT_2',
 'DEstabEnsinoProvPT_3',
 'DCursoProvPT_1',
 'DCursoProvPT_2',
 'DCursoProvPT_3',
 'CursoConcluido_1',
 'CursoConcluido_2',
 'CursoConcluido_3',
 'NotaOuMedia_1',
 'NotaOuMedia_2',
 'NotaOuMedia_3',
 'DescEscala_1',
 'DescEscala_2',
 'DescEscala_3',
 'ID_INDIVIDUO',
 'idcandidato',
 'idgenero',
 'cd_pais',
 'Localidade',
 'nivelInglesCompreensao',
 'nivelInglesFala',
 'nivelInglesEscrita',
 'datacandidaturafim',
 'dprograma',
 'dcurso',
 'DCursoOpcao',
 'idEstadoCandidatura',
 'DEstadoPT',
 'NumberAcademicExperiences',
 'NumberAcademicDegrees',
 'HighestAcademicDegree',
 'Country_1',
 'Country_2',
 'Country_3',
 'Rank',
 'Student Population',
 'Students to Staff Ratio',
 'Overall Score',
 'Teaching',
 'Research Environment',
 'Research Quality',
 'Industry Impact',
 'International Outlook',
 'Percentage_International_Students',
 'Percentage_Female_Stu

In [120]:
df_full.drop(["idCandidatura", "DEstabEnsinoProvPT_1", "DEstabEnsinoProvPT_2", "DEstabEnsinoProvPT_3", "DCursoProvPT_1", "DCursoProvPT_2",
              "DCursoProvPT_3", "CursoConcluido_1", "CursoConcluido_2", "CursoConcluido_3", "NotaOuMedia_1", "NotaOuMedia_2", "NotaOuMedia_3",
              "DescEscala_1", "DescEscala_2", "DescEscala_3", "ID_INDIVIDUO", "idcandidato", "cd_pais", "Localidade", "datacandidaturafim",
              "dcurso", "DCursoOpcao", "Country_1", "Country_2", "Country_3", "MinimoEscala_1", "MaximoEscala_1", "MinimoEscala_2",
              "MaximoEscala_2", "MinimoEscala_3", "MaximoEscala_3", "ScaledGrade_1", "ScaledGrade_2", "ScaledGrade_3", "countries_name",
              "countries_alpha-3", "countries_sub-region", "DataInicio_1", "DataInicio_2", "DataInicio_3", "DataInicio_4", "DataInicio_5",
              "DataInicio_6", "DataFim_1", "DataFim_2", "DataFim_3", "DataFim_4", "DataFim_5", "DataFim_6", "ActivProEntidadePatronal_1",
              "ActivProEntidadePatronal_2", "ActivProEntidadePatronal_3", "ActivProEntidadePatronal_4", "ActivProEntidadePatronal_5",
              "ActivProEntidadePatronal_6", "ActivProFuncao_1", "ActivProFuncao_2", "ActivProFuncao_3", "ActivProFuncao_4", "ActivProFuncao_5",
              "ActivProFuncao_6", "DT_MATRIC", "dataConclusao", "situacaoFinal", "cd_curso"], axis = 1, inplace = True)

print("Number of columns in the dataset:", df_full.shape[1])

df_full.info()

Number of columns in the dataset: 69
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13665 entries, 0 to 13664
Data columns (total 69 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   DGrauAcademicoPT_1                 13665 non-null  object 
 1   DGrauAcademicoPT_2                 3918 non-null   object 
 2   DGrauAcademicoPT_3                 811 non-null    object 
 3   idgenero                           13665 non-null  object 
 4   nivelInglesCompreensao             13665 non-null  object 
 5   nivelInglesFala                    13665 non-null  object 
 6   nivelInglesEscrita                 13665 non-null  object 
 7   dprograma                          13665 non-null  object 
 8   idEstadoCandidatura                13665 non-null  int64  
 9   DEstadoPT                          13665 non-null  object 
 10  NumberAcademicExperiences          13665 non-null  int64  
 11  NumberAcademicDeg

-------
### <font color='#BFD62F'>3.4.2. Encoding Categorical Variables </font> <a class="anchor" id="encoding"></a>
[Back to Contents](#toc)

We now have to make sure that all our features are presented in a numerical format, so that we can use them for modelling (if they are selected). We now have to make a decision regarding how this encoding will be performed:
* `DGrauAcademicoPT_1`, `DGrauAcademicoPT_2`, `DGrauAcademicoPT_3`, and `HighestAcademicDegree` - We can make use of the inherent order of these values to perform ordinal encoding, where an higher value will represent a higher academic degree. Then, we can use these values to add a new variable to our dataset, representing the "average" academic degree. For instance, an applicant with two bachelors' and one master's degree will have a lower "mean" than someone with two masters' and one bachelor's degree.
* `idgenero` - We have three unique values for this variable, and we will transform through a simple label encoding.
* `nivelInglesCompreensao`, `nivelInglesFala`, and `nivelInglesEscrita` - Once again, ordinal encoding will be the solution to transform these variables, which will subsequently allow us to create a "mean" level of the English language throughout these three components.
* `dprograma` - We will transform this column into a binary one, where 0 will represent postgraduate programs and 1 will represent masters' degrees.
* `countries_region` - We will once again use label encoding to transform this attribute, assigning higher values to regions that are farther from Portugal (where Nova IMS is located).
* `datacandidaturafim_time_of_day` - We will be using label encoding, where a higher value will be assigned to applications submitted later throughout the day (we will assume "Night" to follow the "Evening", instead of coming before "Morning").
* `datacandidaturafim_day_of_week` - First, we will do ordinal encoding to transform this variable, with values ranging from 0 to 6 as we move further down the week. Next, we will create additional two attributes, with the sine and cosine representing, as a pair, a unique day of the week. This is because days of the week are cyclical, and this way we can capture, for instance, than Sunday is as close to Saturday as it is to Monday.
* `Program_Name` - Frequency encoding will be our desired way of transforming this variable, since we believe that the popularity of a program (displayed by its frequency in the dataset) may be important when it comes to decide which applicants to admit, and may even impact the grades they are able to attain. We also believe that programs that are more popular share more similarities among themselves, not necessarily due to its contents, but due to they way the program is taught. Although we are aware of the potential data leakage that may arise from the execution of this transformation before splitting the dataset into training and validation, we hope that, this way, no program will be left unseen (which would bring undesired missing values). We are also aware that, once we create a dataset specifically to predict final GPAs (which will be a subset of df_full), frequencies may not exactly be the same - however, we hope that this larger sample will be a better representation of our data, motivating us to adopt the same values.

Note: Since `DEstadoPT` is a target variable, we will leave this column to be handled in the following subchapter.

In [121]:
df_full.head()

,DGrauAcademicoPT_1,DGrauAcademicoPT_2,DGrauAcademicoPT_3,idgenero,nivelInglesCompreensao,nivelInglesFala,nivelInglesEscrita,dprograma,idEstadoCandidatura,DEstadoPT,NumberAcademicExperiences,NumberAcademicDegrees,HighestAcademicDegree,Rank,Student Population,Students to Staff Ratio,Overall Score,Teaching,Research Environment,Research Quality,Industry Impact,International Outlook,Percentage_International_Students,Percentage_Female_Students,Is_Ranked,Previous_IMS_Student,Previous_Data_Studies,Previous_Computer_Studies,Previous_Tech_Studies,Previous_Math_Studies,Previous_Engineering_Studies,Previous_Finance_Studies,Previous_Marketing_Studies,Previous_Field_Studies,Number_Studied_Fields,Non_Finished_Programs,Prop_Finished_Programs,AverageScaledGrade,MaxScaledGrade,MinScaledGrade,countries_region,countries_dist,countries_GDP_per_capita,is_Portuguese,datacandidaturafim_month,datacandidaturafim_day,datacandidaturafim_time_of_day,datacandidaturafim_day_of_week,is_After_Working_Hours,is_Fully_In_Person,Program_Name,Number_Professional_Experiences,Years_of_Experience,Years_of_Longest_Prof_Experience,Working_Upon_Application,Previous_Data_Jobs,Previous_Analytics_Jobs,Previous_Computing_Jobs,Previous_Tech_Jobs,Previous_Math_Jobs,Previous_Engineering_Jobs,Previous_Finance_Jobs,Previous_Marketing_Jobs,Previous_Project_Jobs,Previous_Field_Jobs,Number_Fields_Worked,Previous_Trainee_Jobs,Previous_Jobs_in_Partners,NotaFinal
0,Licenciado,None,None,M,Fluente (QERC: C2),Fluente (QERC: C2),Fluente (QERC: C2),Postgraduate,4,Completa,1,1,Licenciado,1758.0,7044.0,12.1,20.9800,24.1,7.7,28.4,17.0,32.5,6.0,71.0,1,0,0,0,0,0,0,0,0,0,0,0,1.0,0.45,0.45,0.45,Europe,114.3257,27834.773,1,1,8,Afternoon,Monday,0,1,Pós-Graduação em Sistemas de Informação Empresariais,1,0,0.0,0,0,1,0,1,0,0,0,0,0,1,2,0,1,NaN
1,Licenciado,None,None,M,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),Postgraduate,6,Admitido,1,1,Licenciado,2100.0,14080.0,14.8,32.5875,24.3,16.1,53.3,36.0,54.8,16.0,55.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.60,0.60,0.60,Europe,114.3257,27834.773,1,1,9,Evening,Tuesday,0,1,Pós-Graduação em Sistemas de Informação Empresariais,2,1,1.0,1,1,0,0,1,0,0,0,0,0,1,2,0,0,NaN
2,Mestre,None,None,F,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),Postgraduate,4,Completa,1,1,Mestre,1550.0,11773.0,9.7,24.4525,24.3,9.7,31.3,20.9,56.6,17.0,55.0,1,0,0,0,1,0,0,0,0,1,1,0,1.0,0.70,0.70,0.70,Europe,114.3257,27834.773,1,1,15,Evening,Monday,0,1,Pós-Graduação em Sistemas de Informação Empresariais,1,1,1.0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,NaN
3,Licenciado,Mestre,None,M,Nativo ou bilingue,Nativo ou bilingue,Nativo ou bilingue,Postgraduate,6,Admitido,2,2,Mestre,2100.0,14080.0,14.8,32.5875,24.3,16.1,53.3,36.0,54.8,16.0,55.0,0,0,1,0,1,0,0,0,0,1,2,1,0.5,0.70,0.75,0.65,Europe,114.3257,27834.773,1,1,19,Afternoon,Friday,0,1,Pós-Graduação em Geospatial Intelligence,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
4,Licenciado,None,None,M,Fluente (QERC: C2),Avançado (QERC: C1),Intermédio (QERC: B1-B2),Postgraduate,4,Completa,1,1,Licenciado,2100.0,14080.0,14.8,32.5875,24.3,16.1,53.3,36.0,54.8,16.0,55.0,0,0,0,0,0,0,1,0,0,1,1,0,1.0,0.82,0.82,0.82,Europe,114.3257,27834.773,1,1,19,Afternoon,Friday,0,1,Pós-Graduação em Sistemas de Informação Empresariais,3,2,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


#### `DGrauAcademicoPT_1`, `DGrauAcademicoPT_2`, `DGrauAcademicoPT_3`, and `HighestAcademicDegree`

In [122]:
degree_levels = ["Sem Grau", "Ensino Médio", "Finalista", "Licenciado", "Mestre", "Doutorado"]

ordinal_encoder_degree = OrdinalEncoder(categories = [degree_levels], dtype = float, handle_unknown = "use_encoded_value",
                                        unknown_value = np.nan)
 
df_full["DGrauAcademicoPT_1"] = ordinal_encoder_degree.fit_transform(df_full[["DGrauAcademicoPT_1"]].astype(str))
df_full["DGrauAcademicoPT_2"] = ordinal_encoder_degree.fit_transform(df_full[["DGrauAcademicoPT_2"]].astype(str))
df_full["DGrauAcademicoPT_3"] = ordinal_encoder_degree.fit_transform(df_full[["DGrauAcademicoPT_3"]].astype(str))
df_full["HighestAcademicDegree"] = ordinal_encoder_degree.fit_transform(df_full[["HighestAcademicDegree"]].astype(str))

df_full.head()

,DGrauAcademicoPT_1,DGrauAcademicoPT_2,DGrauAcademicoPT_3,idgenero,nivelInglesCompreensao,nivelInglesFala,nivelInglesEscrita,dprograma,idEstadoCandidatura,DEstadoPT,NumberAcademicExperiences,NumberAcademicDegrees,HighestAcademicDegree,Rank,Student Population,Students to Staff Ratio,Overall Score,Teaching,Research Environment,Research Quality,Industry Impact,International Outlook,Percentage_International_Students,Percentage_Female_Students,Is_Ranked,Previous_IMS_Student,Previous_Data_Studies,Previous_Computer_Studies,Previous_Tech_Studies,Previous_Math_Studies,Previous_Engineering_Studies,Previous_Finance_Studies,Previous_Marketing_Studies,Previous_Field_Studies,Number_Studied_Fields,Non_Finished_Programs,Prop_Finished_Programs,AverageScaledGrade,MaxScaledGrade,MinScaledGrade,countries_region,countries_dist,countries_GDP_per_capita,is_Portuguese,datacandidaturafim_month,datacandidaturafim_day,datacandidaturafim_time_of_day,datacandidaturafim_day_of_week,is_After_Working_Hours,is_Fully_In_Person,Program_Name,Number_Professional_Experiences,Years_of_Experience,Years_of_Longest_Prof_Experience,Working_Upon_Application,Previous_Data_Jobs,Previous_Analytics_Jobs,Previous_Computing_Jobs,Previous_Tech_Jobs,Previous_Math_Jobs,Previous_Engineering_Jobs,Previous_Finance_Jobs,Previous_Marketing_Jobs,Previous_Project_Jobs,Previous_Field_Jobs,Number_Fields_Worked,Previous_Trainee_Jobs,Previous_Jobs_in_Partners,NotaFinal
0,3.0,NaN,NaN,M,Fluente (QERC: C2),Fluente (QERC: C2),Fluente (QERC: C2),Postgraduate,4,Completa,1,1,3.0,1758.0,7044.0,12.1,20.9800,24.1,7.7,28.4,17.0,32.5,6.0,71.0,1,0,0,0,0,0,0,0,0,0,0,0,1.0,0.45,0.45,0.45,Europe,114.3257,27834.773,1,1,8,Afternoon,Monday,0,1,Pós-Graduação em Sistemas de Informação Empresariais,1,0,0.0,0,0,1,0,1,0,0,0,0,0,1,2,0,1,NaN
1,3.0,NaN,NaN,M,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),Postgraduate,6,Admitido,1,1,3.0,2100.0,14080.0,14.8,32.5875,24.3,16.1,53.3,36.0,54.8,16.0,55.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.60,0.60,0.60,Europe,114.3257,27834.773,1,1,9,Evening,Tuesday,0,1,Pós-Graduação em Sistemas de Informação Empresariais,2,1,1.0,1,1,0,0,1,0,0,0,0,0,1,2,0,0,NaN
2,4.0,NaN,NaN,F,Avançado (QERC: C1),Avançado (QERC: C1),Avançado (QERC: C1),Postgraduate,4,Completa,1,1,4.0,1550.0,11773.0,9.7,24.4525,24.3,9.7,31.3,20.9,56.6,17.0,55.0,1,0,0,0,1,0,0,0,0,1,1,0,1.0,0.70,0.70,0.70,Europe,114.3257,27834.773,1,1,15,Evening,Monday,0,1,Pós-Graduação em Sistemas de Informação Empresariais,1,1,1.0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,NaN
3,3.0,4.0,NaN,M,Nativo ou bilingue,Nativo ou bilingue,Nativo ou bilingue,Postgraduate,6,Admitido,2,2,4.0,2100.0,14080.0,14.8,32.5875,24.3,16.1,53.3,36.0,54.8,16.0,55.0,0,0,1,0,1,0,0,0,0,1,2,1,0.5,0.70,0.75,0.65,Europe,114.3257,27834.773,1,1,19,Afternoon,Friday,0,1,Pós-Graduação em Geospatial Intelligence,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
4,3.0,NaN,NaN,M,Fluente (QERC: C2),Avançado (QERC: C1),Intermédio (QERC: B1-B2),Postgraduate,4,Completa,1,1,3.0,2100.0,14080.0,14.8,32.5875,24.3,16.1,53.3,36.0,54.8,16.0,55.0,0,0,0,0,0,0,1,0,0,1,1,0,1.0,0.82,0.82,0.82,Europe,114.3257,27834.773,1,1,19,Afternoon,Friday,0,1,Pós-Graduação em Sistemas de Informação Empresariais,3,2,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


Now that the values are encoded, we can create our `AverageAcademicDegree` column and finally drop `DGrauAcademicoPT_1`, `DGrauAcademicoPT_2` and `DGrauAcademicoPT_3`.

In [123]:
df_full["AverageAcademicDegree"] = df_full[["DGrauAcademicoPT_1", "DGrauAcademicoPT_2", "DGrauAcademicoPT_3"]].mean(axis = 1)

df_full.drop(["DGrauAcademicoPT_1", "DGrauAcademicoPT_2", "DGrauAcademicoPT_3"], axis = 1, inplace = True)

#### `idgenero`

We will assign male applicants to the value 0, female students to 1, and those that do not identify with neither of the two genders as 2.

In [124]:
df_full["idgenero"] = df_full["idgenero"].replace({"M": 0, "F": 1, "Other": 2})
df_full["idgenero"].value_counts()

idgenero
0    7642
1    5915
2     108
Name: count, dtype: int64

#### `nivelInglesCompreensao`, `nivelInglesFala`, and `nivelInglesEscrita`

Note from the future: When first trying to encode these variables, we noticed that some of their values were proceeded by blank spaces. Therefore, we should trim these columns before encoding them.

In [125]:
df_full["nivelInglesCompreensao"] = df_full["nivelInglesCompreensao"].str.strip()
df_full["nivelInglesFala"] = df_full["nivelInglesFala"].str.strip()
df_full["nivelInglesEscrita"] = df_full["nivelInglesEscrita"].str.strip()

In [126]:
english_levels = ["Nenhum", "Básico/Elementar (QERC: A1-A2)", "Intermédio (QERC: B1-B2)", "Avançado (QERC: C1)",
                 "Fluente (QERC: C2)", "Nativo ou bilingue"]

ordinal_encoder_english = OrdinalEncoder(categories = [english_levels])
 
df_full["nivelInglesCompreensao"] = ordinal_encoder_english.fit_transform(df_full[["nivelInglesCompreensao"]])
df_full["nivelInglesFala"] = ordinal_encoder_english.fit_transform(df_full[["nivelInglesFala"]])
df_full["nivelInglesEscrita"] = ordinal_encoder_english.fit_transform(df_full[["nivelInglesEscrita"]])

df_full.head()

,idgenero,nivelInglesCompreensao,nivelInglesFala,nivelInglesEscrita,dprograma,idEstadoCandidatura,DEstadoPT,NumberAcademicExperiences,NumberAcademicDegrees,HighestAcademicDegree,Rank,Student Population,Students to Staff Ratio,Overall Score,Teaching,Research Environment,Research Quality,Industry Impact,International Outlook,Percentage_International_Students,Percentage_Female_Students,Is_Ranked,Previous_IMS_Student,Previous_Data_Studies,Previous_Computer_Studies,Previous_Tech_Studies,Previous_Math_Studies,Previous_Engineering_Studies,Previous_Finance_Studies,Previous_Marketing_Studies,Previous_Field_Studies,Number_Studied_Fields,Non_Finished_Programs,Prop_Finished_Programs,AverageScaledGrade,MaxScaledGrade,MinScaledGrade,countries_region,countries_dist,countries_GDP_per_capita,is_Portuguese,datacandidaturafim_month,datacandidaturafim_day,datacandidaturafim_time_of_day,datacandidaturafim_day_of_week,is_After_Working_Hours,is_Fully_In_Person,Program_Name,Number_Professional_Experiences,Years_of_Experience,Years_of_Longest_Prof_Experience,Working_Upon_Application,Previous_Data_Jobs,Previous_Analytics_Jobs,Previous_Computing_Jobs,Previous_Tech_Jobs,Previous_Math_Jobs,Previous_Engineering_Jobs,Previous_Finance_Jobs,Previous_Marketing_Jobs,Previous_Project_Jobs,Previous_Field_Jobs,Number_Fields_Worked,Previous_Trainee_Jobs,Previous_Jobs_in_Partners,NotaFinal,AverageAcademicDegree
0,0,4.0,4.0,4.0,Postgraduate,4,Completa,1,1,3.0,1758.0,7044.0,12.1,20.9800,24.1,7.7,28.4,17.0,32.5,6.0,71.0,1,0,0,0,0,0,0,0,0,0,0,0,1.0,0.45,0.45,0.45,Europe,114.3257,27834.773,1,1,8,Afternoon,Monday,0,1,Pós-Graduação em Sistemas de Informação Empresariais,1,0,0.0,0,0,1,0,1,0,0,0,0,0,1,2,0,1,NaN,3.0
1,0,3.0,3.0,3.0,Postgraduate,6,Admitido,1,1,3.0,2100.0,14080.0,14.8,32.5875,24.3,16.1,53.3,36.0,54.8,16.0,55.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.60,0.60,0.60,Europe,114.3257,27834.773,1,1,9,Evening,Tuesday,0,1,Pós-Graduação em Sistemas de Informação Empresariais,2,1,1.0,1,1,0,0,1,0,0,0,0,0,1,2,0,0,NaN,3.0
2,1,3.0,3.0,3.0,Postgraduate,4,Completa,1,1,4.0,1550.0,11773.0,9.7,24.4525,24.3,9.7,31.3,20.9,56.6,17.0,55.0,1,0,0,0,1,0,0,0,0,1,1,0,1.0,0.70,0.70,0.70,Europe,114.3257,27834.773,1,1,15,Evening,Monday,0,1,Pós-Graduação em Sistemas de Informação Empresariais,1,1,1.0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,NaN,4.0
3,0,5.0,5.0,5.0,Postgraduate,6,Admitido,2,2,4.0,2100.0,14080.0,14.8,32.5875,24.3,16.1,53.3,36.0,54.8,16.0,55.0,0,0,1,0,1,0,0,0,0,1,2,1,0.5,0.70,0.75,0.65,Europe,114.3257,27834.773,1,1,19,Afternoon,Friday,0,1,Pós-Graduação em Geospatial Intelligence,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,3.5
4,0,4.0,3.0,2.0,Postgraduate,4,Completa,1,1,3.0,2100.0,14080.0,14.8,32.5875,24.3,16.1,53.3,36.0,54.8,16.0,55.0,0,0,0,0,0,0,1,0,0,1,1,0,1.0,0.82,0.82,0.82,Europe,114.3257,27834.773,1,1,19,Afternoon,Friday,0,1,Pós-Graduação em Sistemas de Informação Empresariais,3,2,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,3.0


#### `dprograma`

In [127]:
df_full["dprograma"] = df_full["dprograma"].replace({"Postgraduate": 0, "Master": 1})
df_full["dprograma"].value_counts()

dprograma
1    8654
0    5011
Name: count, dtype: int64

#### `countries_region`

In [128]:
print("List of regions:", df_full["countries_region"].unique().tolist())

List of regions: ['Europe', 'Africa', 'Asia', 'Americas', 'Oceania']


In [129]:
df_full["countries_region"] = df_full["countries_region"].replace({"Europe": 0, "Africa": 1, "Asia": 2, "Americas": 3, "Oceania": 4})

#### `datacandidaturafim_time_of_day`

In [130]:
df_full["datacandidaturafim_time_of_day"] = df_full["datacandidaturafim_time_of_day"].replace({"Morning": 0, "Afternoon": 1,
                                                                                               "Evening": 3, "Night": 4})
df_full["datacandidaturafim_time_of_day"].value_counts()

datacandidaturafim_time_of_day
0    5764
1    5397
3    2060
4     444
Name: count, dtype: int64

#### `datacandidaturafim_day_of_week`

In [131]:
days_of_the_week = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]

ordinal_encoder_days = OrdinalEncoder(categories = [days_of_the_week], dtype = float, handle_unknown = "use_encoded_value",
                                        unknown_value = np.nan)
 
df_full["datacandidaturafim_day_of_week"] = ordinal_encoder_days.fit_transform(df_full[["datacandidaturafim_day_of_week"]].astype(str))

df_full["datacandidaturafim_day_of_week_sin"] = np.sin(2 * np.pi * df_full["datacandidaturafim_day_of_week"] / 7)
df_full["datacandidaturafim_day_of_week_cos"] = np.cos(2 * np.pi * df_full["datacandidaturafim_day_of_week"] / 7)

df_full.drop(["datacandidaturafim_day_of_week"], axis = 1, inplace = True)
df_full.head()

,idgenero,nivelInglesCompreensao,nivelInglesFala,nivelInglesEscrita,dprograma,idEstadoCandidatura,DEstadoPT,NumberAcademicExperiences,NumberAcademicDegrees,HighestAcademicDegree,Rank,Student Population,Students to Staff Ratio,Overall Score,Teaching,Research Environment,Research Quality,Industry Impact,International Outlook,Percentage_International_Students,Percentage_Female_Students,Is_Ranked,Previous_IMS_Student,Previous_Data_Studies,Previous_Computer_Studies,Previous_Tech_Studies,Previous_Math_Studies,Previous_Engineering_Studies,Previous_Finance_Studies,Previous_Marketing_Studies,Previous_Field_Studies,Number_Studied_Fields,Non_Finished_Programs,Prop_Finished_Programs,AverageScaledGrade,MaxScaledGrade,MinScaledGrade,countries_region,countries_dist,countries_GDP_per_capita,is_Portuguese,datacandidaturafim_month,datacandidaturafim_day,datacandidaturafim_time_of_day,is_After_Working_Hours,is_Fully_In_Person,Program_Name,Number_Professional_Experiences,Years_of_Experience,Years_of_Longest_Prof_Experience,Working_Upon_Application,Previous_Data_Jobs,Previous_Analytics_Jobs,Previous_Computing_Jobs,Previous_Tech_Jobs,Previous_Math_Jobs,Previous_Engineering_Jobs,Previous_Finance_Jobs,Previous_Marketing_Jobs,Previous_Project_Jobs,Previous_Field_Jobs,Number_Fields_Worked,Previous_Trainee_Jobs,Previous_Jobs_in_Partners,NotaFinal,AverageAcademicDegree,datacandidaturafim_day_of_week_sin,datacandidaturafim_day_of_week_cos
0,0,4.0,4.0,4.0,0,4,Completa,1,1,3.0,1758.0,7044.0,12.1,20.9800,24.1,7.7,28.4,17.0,32.5,6.0,71.0,1,0,0,0,0,0,0,0,0,0,0,0,1.0,0.45,0.45,0.45,0,114.3257,27834.773,1,1,8,1,0,1,Pós-Graduação em Sistemas de Informação Empresariais,1,0,0.0,0,0,1,0,1,0,0,0,0,0,1,2,0,1,NaN,3.0,0.781831,0.623490
1,0,3.0,3.0,3.0,0,6,Admitido,1,1,3.0,2100.0,14080.0,14.8,32.5875,24.3,16.1,53.3,36.0,54.8,16.0,55.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.60,0.60,0.60,0,114.3257,27834.773,1,1,9,3,0,1,Pós-Graduação em Sistemas de Informação Empresariais,2,1,1.0,1,1,0,0,1,0,0,0,0,0,1,2,0,0,NaN,3.0,0.974928,-0.222521
2,1,3.0,3.0,3.0,0,4,Completa,1,1,4.0,1550.0,11773.0,9.7,24.4525,24.3,9.7,31.3,20.9,56.6,17.0,55.0,1,0,0,0,1,0,0,0,0,1,1,0,1.0,0.70,0.70,0.70,0,114.3257,27834.773,1,1,15,3,0,1,Pós-Graduação em Sistemas de Informação Empresariais,1,1,1.0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,NaN,4.0,0.781831,0.623490
3,0,5.0,5.0,5.0,0,6,Admitido,2,2,4.0,2100.0,14080.0,14.8,32.5875,24.3,16.1,53.3,36.0,54.8,16.0,55.0,0,0,1,0,1,0,0,0,0,1,2,1,0.5,0.70,0.75,0.65,0,114.3257,27834.773,1,1,19,1,0,1,Pós-Graduação em Geospatial Intelligence,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,3.5,-0.974928,-0.222521
4,0,4.0,3.0,2.0,0,4,Completa,1,1,3.0,2100.0,14080.0,14.8,32.5875,24.3,16.1,53.3,36.0,54.8,16.0,55.0,0,0,0,0,0,0,1,0,0,1,1,0,1.0,0.82,0.82,0.82,0,114.3257,27834.773,1,1,19,1,0,1,Pós-Graduação em Sistemas de Informação Empresariais,3,2,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,3.0,-0.974928,-0.222521


#### `Program_Name`

In [132]:
program_frequencies = df_full["Program_Name"].value_counts(normalize = True)
df_full["Program_Name"] = df_full["Program_Name"].map(program_frequencies)

df_full["Program_Name"].value_counts().head()

Program_Name
0.207464    2835
0.177387    2424
0.138383    1891
0.077717    1062
0.071716     980
Name: count, dtype: int64

#### Optimizing Data Types

Finally, we can optimize the memory usage of our dataset by reducing data types to their possible minimum, allowing our models (and remaning code) to run more effectively. For that purpose, we will adapt a function from [Stack Overflow](https://stackoverflow.com/questions/57856010/automatically-optimizing-pandas-dtypes).

In [133]:
print("Memory usage before optimizing data types:", df_full.memory_usage(deep = True).sum() / 1e6)
df_full = tf.auto_opt_pd_dtypes(df_full)
print("Memory usage after optimizing data types:", df_full.memory_usage(deep = True).sum() / 1e6)

Memory usage before optimizing data types: 7.900896
Memory usage after optimizing data types: 3.049821


We reduced memory usage by around 60%, a considerable improvement that will surely have a positive impact going forward.

We are now ready to set up the datasets to be used in our specific preprocessing and modeling tasks.

-------
### <font color='#BFD62F'>3.4.3. Splitting the Dataframes </font> <a class="anchor" id="splitting"></a>
[Back to Contents](#toc)

Our classification and regression tasks will require different datasets with different attributes. Therefore, it is now time to create two different tables that will be the source of data for each case going forward.

While, for the admissions, all applicants that are present in the dataset are eligible, the same does not occur for the prediction of final GPA, as we are only interested in those who have a value for `NotaFinal`.

In [134]:
df_admissions = df_full.drop(columns = ["NotaFinal"])
df_grades = df_full.drop(columns = ["idEstadoCandidatura", "DEstadoPT"]).dropna(subset = ["NotaFinal"])

#### Transforming Admissions' Target

In the exploration stage, we learned that we could have up to 11 unique values for our target, `DEstadoPT`. Considering that we do not want to predict for as many as 11 categories, we should transform these values into more meaningful groups.

In [135]:
df_admissions["DEstadoPT"].unique().tolist()

['Completa',
 'Admitido',
 'Em Análise',
 'Excluido',
 'Desistiu após seleção',
 'Não Admitido',
 'Desistiu',
 'Incompleta',
 'Lista Espera',
 'Não efetuou pagamento',
 'A pagamento']

In [136]:
df_admissions["DEstadoPT"].value_counts()

DEstadoPT
Admitido                 7855
Desistiu após seleção    2862
Não Admitido             2622
Completa                  137
Desistiu                   82
Lista Espera               49
Excluido                   23
A pagamento                17
Não efetuou pagamento      10
Em Análise                  4
Incompleta                  4
Name: count, dtype: int64

We will assume the following:
* __Admitted students__ - Students with one of the following values: "Admitido", "Desistiu após seleção", and "Completa".
* __Rejected students__ - Students with one of the following values: "Não Admitido", "Lista Espera", and "Excluido".

We will drop students who have the values of "Desistiu", "A pagamento", "Não efetuou pagamento", "Incompleta", and "Em Análise", since it is difficult to know for sure whether they were accepted or not.

In [137]:
df_admissions = df_admissions[~df_admissions["DEstadoPT"].isin(["Desistiu", "A pagamento", "Não efetuou pagamento", "Incompleta", "Em Análise"])]

admitted_status = ["Admitido", "Desistiu após seleção", "Completa"]
df_admissions["Admitted"] = df_admissions["DEstadoPT"].apply(lambda x: 1 if x in admitted_status else 0)
df_admissions.groupby("Admitted")["DEstadoPT"].value_counts()

Admitted  DEstadoPT            
0         Não Admitido             2622
          Lista Espera               49
          Excluido                   23
1         Admitido                 7855
          Desistiu após seleção    2862
          Completa                  137
Name: count, dtype: int64

We can now drop the `idEstadoCandidatura` and `DEstadoPT` columns.

In [138]:
df_admissions.drop(columns = ["idEstadoCandidatura", "DEstadoPT"], inplace = True)

#### Assessing Target Distributions

After all these transformations, we can now finally assess our applicants and students' population is distributed for both target variables.

In [139]:
df_admissions_target = df_admissions["Admitted"].value_counts().rename({0: "No", 1: "Yes"}).to_frame(name = "Absolute Frequency")
df_admissions_target["Relative Frequency"] = (df_admissions["Admitted"].value_counts(normalize = True) * 100).rename({0: "No", 1: "Yes"}).values
df_admissions_target["Relative Frequency"] = df_admissions_target["Relative Frequency"].map("{:.2f}%".format)

df_admissions_target

,Absolute Frequency,Relative Frequency
Admitted,,
Yes,10854,80.12%
No,2694,19.88%


As we already expected from our earlier insights and transformations, we identify the the presence of a larger group of admitted applicants, compared to those that were rejected. This suggests a potential benefit of resampling techniques in order to increase the model's predictive ability (although we will only confirm this theory at the end of our preprocessing).

In [140]:
tf.generate_histogram(df = df_grades,
                      column = "NotaFinal",
                      title = "Final GPA Histogram",
                      xaxis_label = "Final GPA",
                      bin_size = 10)

Similarly to our initial analysis, our population's final GPAs resembles a normal distribution, centered around its mean of around 16-16.5. The are no negative grades (<10), but there are also only 3 students who completed their program with a GPA above 19.5.

#### Training and Validation Datasets

Our last step before heading towards more specific preprocessing will be to split both datasets to create training and validation data. Although this should, theoretically, have been done before any preprocessing was made, the only impacted steps were the filling of the missing values of HEIs data and the frequency encoding of `dprograma`, where a conscious and justified decision was taken and explained for performing them before the split.

For our study, the __hold-out__ method will be the preferred option for creating training and validation datasets.

In both cases, we will leave 30% of the data for validation. For the admissions' table, we will stratify our division so that each target class is equally represented in both datasets. For the grades' table, we will set the "shuffle" parameter to False, so that the most recent observations (which come later in the dataset) are used for validation.

In [141]:
X_admissions = df_admissions.drop(columns = ["Admitted"])
y_admissions = df_admissions["Admitted"]

X_admissions_train, X_admissions_val, y_admissions_train, y_admissions_val = train_test_split(
    X_admissions, y_admissions, test_size = 0.3, random_state = 92, stratify = y_admissions)

X_admissions_train.shape, X_admissions_val.shape, y_admissions_train.shape, y_admissions_val.shape

((9483, 65), (4065, 65), (9483,), (4065,))

In [142]:
X_grades = df_grades.drop(columns = ["NotaFinal"])
y_grades = df_grades["NotaFinal"]

X_grades_train, X_grades_val, y_grades_train, y_grades_val = train_test_split(
    X_grades, y_grades, test_size = 0.3, random_state = 92, shuffle = False)

X_grades_train.shape, X_grades_val.shape, y_grades_train.shape, y_grades_val.shape

((2254, 65), (966, 65), (2254,), (966,))

## <font color='#BFD62F'>3.5. Storing the Dataframes</font> <a class="anchor" id="storing"></a>
[Back to Contents](#toc)

We will store the dataframes that are going to be used in the next notebooks, starting with those dedicated to the specific preprocessing steps for the tasks of predicting admissions and final GPAs. See you there! 👋

In [143]:
%store df_admissions
%store df_grades

%store X_admissions_train
%store X_admissions_val
%store y_admissions_train
%store y_admissions_val

%store X_grades_train
%store X_grades_val
%store y_grades_train
%store y_grades_val

Stored 'df_admissions' (DataFrame)
Stored 'df_grades' (DataFrame)
Stored 'X_admissions_train' (DataFrame)
Stored 'X_admissions_val' (DataFrame)
Stored 'y_admissions_train' (Series)
Stored 'y_admissions_val' (Series)
Stored 'X_grades_train' (DataFrame)
Stored 'X_grades_val' (DataFrame)
Stored 'y_grades_train' (Series)
Stored 'y_grades_val' (Series)
